# Week 1

### Caesar cipher

In [ ]:
def encrypt(message, key):
  encrypted_msg = ""
  for i in message:
    if (i.islower()):
      '''
        ord(i) - 97 => for getting position (0 - 25) of alphabet from ascii values 
        (ord(i)) - 97 + key => shift by key positions
        ((ord(i) - 97) + key) % 26 => if shifting by key positions exceeds 25 (to get a position within 0 to 25)
        ((ord(i) - 97 + key) % 26) + 97 => add 97 back to get ascii value
      '''
      position = ((ord(i) - 97 + key) % 26) + 97
    else:
      position = ((ord(i) - 65 + key) % 26) + 65
    encrypted_msg += chr(position)
  return encrypted_msg

def decrypt(message, key):
  decrypted_msg = ""
  for i in message:
    if (i.islower()):
      position = ((ord(i) - 97 - key) % 26) + 97
    else:
      position = ((ord(i) - 65 - key) % 26) + 65
    decrypted_msg += chr(position)
  return decrypted_msg

message = input('Enter message to be passed:\n')
key = int(input('Enter key (shift):\n'))
a = encrypt(message, key)
print('Encrypted message:', a)
b = decrypt(a, key)
print('Decrypted message:', b)

Enter message to be passed:
sample
Enter key (shift):
3
Encrypted message: vdpsoh
Decrypted message: sample


### Playfair cipher

In [ ]:
n = 5
def fillTable(key):
  i, j, k = -1, -1, 0
  alpha = []
  for p in range(26):
    alpha.append(chr(p + 97))
  for p1 in range(n):
    p2 = 0
    while(p2 < n):
      if (len(key) <= k):
        i = p1
        j = p2
        break
      if key[k] in alpha:
        table[p1][p2] = key[k]
        alpha.remove(key[k])
        p2 += 1
      k += 1
    if (i != -1):
      break
  k = 0
  # print(i, j)
  for p1 in range(i, n):
    for p2 in range(j, n):
      table[p1][p2] = alpha[k]
      k += 1
    j = 0
  table[n - 1][n - 1] += alpha[len(alpha) - 1]
  # print(table)

def add_bogus(message):
  i = 0
  while (i < (len(message) - 1)):
    if (message[i] == message[i + 1]):
      message = message[:i + 1] + 'x' + message[i + 1:]
      i += 1
    i += 1
  return message

def getRowColumn(k):
  for i in range(n):
    for j in range(n):
      if (k in table[i][j]):
        return [i, j]
  return [-1, -1]


def encrypt(message):
  i = 0
  encrypted = ""
  while (i < len(message) - 1):
    c1 = getRowColumn(message[i])
    c2 = getRowColumn(message[i + 1])
    # print(message[i], c1, c2)
    if (c1[1] == c2[1]):
      b1 = (c1[0] + 1) % n
      b2 = (c2[0] + 1) % n
      encrypted += table[b1][c1[1]] + table[b2][c2[1]]
    elif (c1[0] == c2[0]):
      b1 = (c1[1] + 1) % n
      b2 = (c2[1] + 1) % n
      encrypted += table[c1[0]][b1] + table[c2[0]][b2]
    else:
      encrypted += table[c1[0]][c2[1]] + table[c2[0]][c1[1]]
    i += 2
  if (len(message) % 2 != 0):
    encrypted += message[len(message) - 1]
  return encrypted

def decrypt(message):
  i = 0
  decrypted = ""
  while (i < len(message) - 1):
    c1 = getRowColumn(message[i])
    c2 = getRowColumn(message[i + 1])
    if (c1[1] == c2[1]):
      b1 = (c1[0] - 1) % n
      b2 = (c2[0] - 1) % n
      decrypted += table[b1][c1[1]] + table[b2][c2[1]]
    elif (c1[0] == c2[0]):
      b1 = (c1[1] - 1) % n
      b2 = (c2[1] - 1) % n
      decrypted += table[c1[0]][b1] + table[c2[0]][b2]
    else:
      decrypted += table[c1[0]][c2[1]] + table[c2[0]][c1[1]]
    i += 2
  if (len(message) % 2 != 0):
    decrypted += message[len(message) - 1]
  return decrypted


table = []
for i in range(n):
  table.append([0] * n)

key = input("Enter key:\n")
fillTable(key)

message = input("Enter message:\n")
message = add_bogus(message)
encrypted = encrypt(message)
print('Encrypted message:', encrypted)

print('Decrypted message:', decrypt(encrypted))

Enter key:
key
Enter message:
sample
Encrypted message: rbjrjy
Decrypted message: sample


### Vigenere cipher

In [ ]:
def getKey(message, key):
	if len(message) == len(key):
		return(key)
	else:
		for i in range(len(message) - len(key)):
			key += key[i % len(key)]
	return key
	
def encrypt(message, key):
	encrypted = ""
	for i in range(len(message)):
		position = (ord(message[i]) + ord(key[i])) % 26
		position += 65
		encrypted += (chr(position))
	return encrypted
	
def decrypt(message, key):
	decrypted = ""
	for i in range(len(message)):
		position = (ord(message[i]) - ord(key[i]) + 26) % 26
		position += 65
		decrypted += chr(position)
	return decrypted
	
message = input("Enter message:\n")
key = input("Enter key:\n")
key = getKey(message, key)
encrypted = encrypt(message, key)
print("Encrypted message:", encrypted)
print("Decrypted message:", decrypt(encrypted, key))

Enter message:
sample
Enter key:
key
Encrypted message: OQWLBO
Decrypted message: YGSVRK


# Week 2

### Attacks on Affine cipher

In [ ]:
from sympy import symbols, Eq, solve

alpha = {}
for i in range(26):
  alpha[i] = chr(97 + i)

rev_alpha = {}
for i in range(26):
  rev_alpha[chr(97 + i)] = i

def encrypt(original, a, b):
  cipher = ''
  for i in original:
    cipher += alpha[(a * rev_alpha[i] + b) % 26]
  return cipher

def multiplicativeInverse(a):
  d, p, q, r, t1, t2, t = 0, 26, a, 0, 0, 1, 0

  while (q > 0):
    d = p // q
    r = p % q
    t = t1 - d * t2

    p = q
    q = r

    t1 = t2
    t2 = t

  if (t1 < 0):
    return t1 + 26
  return t1

possible_a = [3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]
possible_b = []
possible_plainTexts = {}
for i in range(26):
  possible_b.append(i)

def bruteforce(cipher):
  for a in possible_a:
    for b in possible_b:
      plain = ''
      for c in cipher:
        a_inverse = multiplicativeInverse(a)
        plain += alpha[((rev_alpha[c] - b) * a_inverse) % 26]
      possible_plainTexts[plain] = (a, b)
  
  if original in possible_plainTexts:
    print('BRUTEFORCE - Key used for encryption', possible_plainTexts[original])


def known_plain_text_attack(cipher, plain):
  if (len(cipher) < 2):
    print('Not possible to perform Known Plain Text Attack on affine cipher, if the cipher text length is less than 2.')
    return

  a, b = symbols('a,b')
  i = 0
  notFound = True
  while (notFound):
    eq1 = Eq((rev_alpha[plain[0]]*a + b), i * 26 + rev_alpha[cipher[0]])

    for j in range(i + 1):
      eq2 = Eq((rev_alpha[plain[1]]*a + b), j * 26 + rev_alpha[cipher[1]])

      d = solve((eq1, eq2), (a, b))
      if (int(d[a]) == d[a] and int(d[b]) == d[b]):
        while (d[a] < 0):
          d[a] += 26
        while (d[b] < 0):
          d[b] += 26
        print('KNOWN PLAIN TEXT ATTACK - Key used for encryption', d)
        notFound = False
        break
    i += 1

original = input('Enter plain text:\n')
a = int(input("Enter 'a' in key:\n"))
while (a not in possible_a):
  a = int(input("Invalid. 'a' must be relatively prime to 26:"))

b = int(input("Enter 'b' in key:\n"))
while (b not in possible_b):
  b = int(input("Invalid. 'b' must be in between 0 and 25: "))

cipher = encrypt(original, a, b)
print('Cipher text:', cipher, '\n\n')
bruteforce(cipher)
known_plain_text_attack(cipher, original)


Enter plain text:
sample
Enter 'a' in key:
5
Enter 'b' in key:
4
Cipher text: qembhy 


BRUTEFORCE - Key used for encryption (5, 4)
KNOWN PLAIN TEXT ATTACK - Key used for encryption {a: 5, b: 4}


### Attacks on Additive cipher

In [ ]:
def encrypt(message, key):
  encrypted_msg = ""
  for i in message:
    if (i.islower()):
      '''
        ord(i) - 97 => for getting position (0 - 25) of alphabet from ascii values 
        (ord(i)) - 97 + key => shift by key positions
        ((ord(i) - 97) + key) % 26 => if shifting by key positions exceeds 25 (to get a position within 0 to 25)
        ((ord(i) - 97 + key) % 26) + 97 => add 97 back to get ascii value
      '''
      position = ((ord(i) - 97 + key) % 26) + 97
    else:
      position = ((ord(i) - 65 + key) % 26) + 65
    encrypted_msg += chr(position)
  return encrypted_msg

def decrypt(cipher, key):
  decrypted_msg = ""
  for i in cipher:
    if (i.islower()):
      position = ((ord(i) - 97 - key) % 26) + 97
    else:
      position = ((ord(i) - 65 - key) % 26) + 65
    decrypted_msg += chr(position)
  return decrypted_msg

def bruteforce(cipher):
  possible_plainTexts = {}
  for key in range(26):
    possible_plainTexts[decrypt(cipher, key)] = key
  if plaintext in possible_plainTexts:
    print('BRUTEFORCE - Key used:', possible_plainTexts[plaintext])

plaintext = input('Enter plain text:\n')
key = int(input('Enter key (shift):\n'))
cipher = encrypt(plaintext, key)
print('Encrypted message:', cipher)
bruteforce(cipher)

Enter plain text:
sample
Enter key (shift):
4
Encrypted message: weqtpi
BRUTEFORCE - Key used: 4


### Attacks on Transposition cipher

In [ ]:
# ROW TRANSPOSITION CIPHER
from itertools import permutations

columns = 5

def encrypt(plain, key):
  p = 0
  rectangle = []
  i, j = 0, 0
  while (i < len(plain)):
    l = []
    for j in range(columns):
      if (i >= len(plain)):
        break
      l.append(plain[i])
      i += 1
    while (len(l) < columns):
      l.append(chr(p + 97))
      p += 1
    rectangle.append(l)
  
  cipher = ''
  for i in key:
    for k in rectangle:
      cipher += k[int(i) - 1]

  return cipher

def possibleKeys(key):
  keys = []
  permList = permutations(key)
  for perm in list(permList):
    keys.append(''.join(perm))
  return keys

def bruteforce(cipher):
  keys = possibleKeys('12345')
  # keys = ['32145']
  li = []
  rows = len(cipher) // 5
  for i in range(0, len(cipher), rows):
    li.append(cipher[i:i + rows])
  possible_plainTexts = {}
  for key in keys:
    plain = ''
    for p1 in range(len(li[0])):
      for i in key:
        plain += li[int(i) - 1][p1]
    possible_plainTexts[plain[:len(plain_text)]] = (key, plain)

  # indicates plain text is found in possible plain texts
  if plain_text in possible_plainTexts:
    print('\nOriginal plain text is found using brutforce attack.')
    print('Key used:', possible_plainTexts[plain_text][0], '\nPlain text:', plain_text)
    print('Original plain text with padded alphabets:', possible_plainTexts[plain_text][1])


plain_text = input('Enter plain text:\n')
key = input('Enter key:\n')
cipher = encrypt(plain_text, key)
print('Encrypted message:', cipher)
bruteforce(cipher)

Enter plain text:
sample
Enter key:
23145
Encrypted message: aambsepcld

Original plain text is found using brutforce attack.
Key used: 31245 
Plain text: sample
Original plain text with padded alphabets: sampleabcd


# Week 3

### RC4

In [ ]:
key_in = input('Enter KEY:\n')
plainText = input('Enter PLAIN TEXT:\n')
def rc4(key_in, plainText):
  bitss = ''
  keyy = ''
  if (len(key_in) < 256):
    repeat = 256 // len(key_in)
    key_in *= repeat
    i = 0
    while (len(key_in) < 256):
      key_in += key_in[i]
      i += 1
  S = [-1] * 256
  K = [-1] * 256
  for i in range(256):
    S[i] = i
    K[i] = ord(key_in[i])
  j = 0
  for i in range(256):
    j = (j + S[i] + K[i]) % 256
    temp = S[i]
    S[i] = S[j]
    S[j] = temp

  i, j, p1 = 0, 0, 0
  textt = ''

  while (p1 < len(plainText)):
    i = (i + 1) % 256
    j = (j + S[i]) % 256
    temp = S[i]
    S[i] = S[j]
    S[j] = temp

    t = (S[i] + S[j]) % 256

    key_stream_byte = S[t]
    keyy += "{0:08b}".format(S[t])

    c = int("{0:08b}".format(ord(plainText[p1])), 2) ^ int("{0:08b}".format(key_stream_byte), 2)

    bitss += '{0:08b}'.format(c)
    textt += chr(int('{0:08b}'.format(c), 2))

    p1 += 1

  print('KEY', keyy)
  return bitss, textt

cipherBits, cipherText = rc4(key_in, plainText)
print('Cipher Text:', cipherText)
print('Encrypted (bits):', cipherBits)
plainBits, plain = rc4(key_in, cipherText)
print('Plain text:', plain)
print('Decrypted (bits):', plainBits)

Enter KEY:
key
Enter PLAIN TEXT:
sample
KEY 000010110110110000110100111011010010010010001111
YHê
Encrypted (bits): 011110000000110101011001100111010100100011101010
KEY 000010110110110000110100111011010010010010001111
Plain text: sample
Decrypted (bits): 011100110110000101101101011100000110110001100101


### Cryptanalysis of RC4

In [ ]:
def rc4(key_in, plainText):
  bitss = ''
  keyy = ''
  if (len(key_in) < 256):
    repeat = 256 // len(key_in)
    key_in *= repeat
    i = 0
    while (len(key_in) < 256):
      key_in += key_in[i]
      i += 1
  S = [-1] * 256
  K = [-1] * 256
  for i in range(256):
    S[i] = i
    K[i] = ord(key_in[i])
  j = 0
  for i in range(256):
    j = (j + S[i] + K[i]) % 256
    temp = S[i]
    S[i] = S[j]
    S[j] = temp

  i, j, p1 = 0, 0, 0
  textt = ''

  while (p1 < len(plainText)):
    i = (i + 1) % 256
    j = (j + S[i]) % 256
    temp = S[i]
    S[i] = S[j]
    S[j] = temp

    t = (S[i] + S[j]) % 256

    key_stream_byte = S[t]
    keyy += "{0:08b}".format(S[t])

    c = int("{0:08b}".format(ord(plainText[p1])), 2) ^ int("{0:08b}".format(key_stream_byte), 2)

    bitss += '{0:08b}'.format(c)
    textt += chr(int('{0:08b}'.format(c), 2))

    p1 += 1

  return bitss, textt

key_in = "key"
plainText = "prasuna"

cipherBits, cipherText = rc4(key_in, plainText)
plainBits, plain = rc4(key_in, cipherText)

'''
let's say we have a plain text and cipher text, then using them we can get the key and use it to decrypt other cipher texts
'''

def getKey(cipher, plain):
  key = int(cipher, 2) ^ int(plain, 2)
  key = "{0:0{1}b}".format(key, len(cipher))

  return key

def decrypt(cipher, key):
  plain = int(cipher, 2) ^ int(key, 2)
  plain = "{0:0{1}b}".format(plain, len(cipher))

  i = 0
  plainText = ''
  while (i < len(plain)):
    plainText += chr(int(plain[i:i + 8], 2))
    i += 8
  
  return plainText

key = getKey(cipherBits, plainBits)
print('CHOSEN PLAIN TEXT ATTACK / CHOSEN CIPHER TEXT ATTACK')
print('---works only if the plain text length is same as key length (cracked)---')

print('\nKEY cracked using plain text', plainBits, 'and cipher text', cipherBits + ':\n' + key)

plainText1 = "holiday"
cipherBits, cipherText = rc4(key_in, plainText1)
plain = decrypt(cipherBits, key)
print('Plain text from cipher:', plain)

possibleKeys = []
def allPossibleKeys(i, string, n):
  if n == len(string):
    possibleKeys.append(string)
    return
  s = string + '0'
  allPossibleKeys(i + 1, s, n)

  s1 = string + '1'
  allPossibleKeys(i + 1, s1, n)

def bruteforce(cipher):
  allPossibleKeys(0, '', len(cipher))

  possiblePlainTexts = {}

  for key in possibleKeys:
    possiblePlainTexts[decrypt(cipher, key)] = key
  
  return possiblePlainTexts

plainText1 = "a"
cipherBits, cipherText = rc4(key_in, plainText1)

possiblePlainTexts = bruteforce(cipherBits)
if plainText1 in possiblePlainTexts:
  print('\nCracked through BRUTE FORCE\nPlain Text --', plainText1, '\nKey --', possiblePlainTexts[plainText1])


CHOSEN PLAIN TEXT ATTACK / CHOSEN CIPHER TEXT ATTACK
---works only if the plain text length is same as key length (cracked)---

KEY cracked using plain text 01110000011100100110000101110011011101010110111001100001 and cipher text 01111011000111100101010110011110010100011110000100011010:
00001011011011000011010011101101001001001000111101111011
Plain text from cipher: holiday

Cracked through BRUTE FORCE
Plain Text -- a 
Key -- 00001011


# Week 4

### DES

In [ ]:
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]
 

exp_pbox = [32, 1, 2 , 3 , 4 , 5 , 4 , 5,
         6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
         12, 13, 12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27,
         28, 29, 28, 29, 30, 31, 32, 1 ]
         
 
straight_per = [ 16,  7, 20, 21,
        29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2,  8, 24, 14,
        32, 27,  3,  9,
        19, 13, 30,  6,
        22, 11,  4, 25 ]

sbox =  [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
          [ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
          [ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
          [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ]],
            
         [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
            [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
            [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
           [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]],
   
         [ [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
           [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
           [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ]],
       
          [ [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
           [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
           [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
            [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14] ],
        
          [ [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
           [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
            [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
           [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ]],
       
         [ [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
           [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
            [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
            [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13] ],
         
          [ [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
           [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
            [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
            [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12] ],
        
         [ [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
            [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
            [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
            [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11] ] ]
   
# Final Permutation Table
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
               39, 7, 47, 15, 55, 23, 63, 31,
               38, 6, 46, 14, 54, 22, 62, 30,
               37, 5, 45, 13, 53, 21, 61, 29,
               36, 4, 44, 12, 52, 20, 60, 28,
               35, 3, 43, 11, 51, 19, 59, 27,
               34, 2, 42, 10, 50, 18, 58, 26,
               33, 1, 41, 9, 49, 17, 57, 25 ]

keyparity = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

shift_table = [1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]
 
key_comp = [14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]


size = 64
def initialPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[initial_perm[i] - 1]
  
  return ''.join(li)

def finalPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[final_perm[i] - 1]
  
  return ''.join(li)

def expandPBox(in1):
  expanded = ''
  for i in range(len(exp_pbox)):
    expanded += str(in1[exp_pbox[i] - 1])

  return expanded

def s_box(in1, num):
  row = in1[0] + in1[5]
  col = in1[1 : 5]
  curr_sbox = sbox[num]
  row = curr_sbox[int(row, 2)]
  entry = row[int(col, 2)]
  return "{0:04b}".format(entry)

def compressBox(in1):
  res = ''
  for i in range(0, 48, 6):
    res += s_box(in1[i:i + 6], i // 6)
    # print('res', len(res), res)
  return res

def straightPBox(in1):
  li = [-1] * 32
  for i in range(len(straight_per)):
    # print(straight_per[i], len(in1))
    li[i] = in1[straight_per[i] - 1]
  
  return ''.join(li)

def roundFunction(right, key):
  in1 = expandPBox(right)
  xor_res = xor(in1, key)
  compressed = compressBox(xor_res)
  # print('---', len(compressed))
  res = straightPBox(compressed)

  return res

def round(left, right, key):
  next_l = right
  inp = roundFunction(right, key)
  next_r = xor(left, inp)

  return next_l, next_r

def xor(in1, in2):
  res = ''
  for i in range(len(in1)):
    res += str(int(in1[i]) ^ int(in2[i]))

  return res

plaintext = ''
import random
for i in range(size):
  plaintext += str(random.randint(0, 1))

def dropParity(key):
  k = ''
  for i in range(len(keyparity)):
    k += key[keyparity[i] - 1]

  return k

def shift_left(val, pos):
    s = ''
    for i in range(pos):
        for j in range(1, len(val)):
            s = s + val[j]
        s = s + val[0]
        val = s
        s = ''
    return val

def roundKeyGen(key):
  left = key[0:28]
  right = key[28:56]
  
  roundKeys  = []
  for i in range(0, 16):
      left = shift_left(left, shift_table[i])
      right = shift_left(right, shift_table[i])
      
      keyy = left + right

      round_key = ''
      for i in range(len(key_comp)):
        round_key += keyy[key_comp[i] - 1]
      
      roundKeys.append(round_key)
    
  # print('RKB\n\n', roundKeys, '\n\n\n')
  return roundKeys

def encrypt(plaintext, roundKeys):
  afterInitial = initialPermutation(plaintext)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  cipherText = finalPermutation(afterAllRounds)

  return cipherText

def decrypt(cipher, roundKeys):
  afterInitial = initialPermutation(cipher)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  plain = finalPermutation(afterAllRounds)

  return plain

def bin2hex(s):
    mp = {"0000" : '0',
          "0001" : '1',
          "0010" : '2',
          "0011" : '3',
          "0100" : '4',
          "0101" : '5',
          "0110" : '6',
          "0111" : '7',
          "1000" : '8',
          "1001" : '9',
          "1010" : 'A',
          "1011" : 'B',
          "1100" : 'C',
          "1101" : 'D',
          "1110" : 'E',
          "1111" : 'F' }
    hex = ""
    for i in range(0,len(s),4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
         
    return hex

def hex2bin(s):
    mp = {'0' : "0000",
          '1' : "0001",
          '2' : "0010",
          '3' : "0011",
          '4' : "0100",
          '5' : "0101",
          '6' : "0110",
          '7' : "0111",
          '8' : "1000",
          '9' : "1001",
          'A' : "1010",
          'B' : "1011",
          'C' : "1100",
          'D' : "1101",
          'E' : "1110",
          'F' : "1111" }
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

plaintext = hex2bin(input('Enter plain text in hexa-decimal format:\n'))
key = hex2bin(input('Enter key in hexa-decimal format:'))
key = dropParity(key)
roundKeys = roundKeyGen(key)
print('\nENCRYPTION:')
cipher = encrypt(plaintext, roundKeys)
print('Cipher Text:',bin2hex(cipher))
print('\nDECRYPTION:')
plain = decrypt(cipher, roundKeys[::-1])
print('Plain Text:', bin2hex(plain))1

Enter plain text in hexa-decimal format:
ABCD123456789123
Enter key in hexa-decimal format:ABCD456789ABCD12

ENCRYPTION:
Round 1   KEY: 9A4A179BC139 Text: 43A92395B7B113A6
Round 2   KEY: 6D4E43A54EAE Text: B7B113A6F15DEA0A
Round 3   KEY: 63F18C1C1AD3 Text: F15DEA0A45F9764C
Round 4   KEY: 1885D397C075 Text: 45F9764C5B81AF9B
Round 5   KEY: 75483B03AFC0 Text: 5B81AF9BC9D1EC6F
Round 6   KEY: A7A184B8A515 Text: C9D1EC6F03C2D0B5
Round 7   KEY: 1A0E976B4686 Text: 03C2D0B5D1138CC6
Round 8   KEY: 7D30385C618B Text: D1138CC6F0CB1253
Round 9   KEY: 41F9AC34AD35 Text: F0CB1253B8FEE3D4
Round 10   KEY: 90A5D36B0CD2 Text: B8FEE3D4507A682D
Round 11   KEY: 754E234DE11B Text: 507A682D8F667981
Round 12   KEY: E3B18427544C Text: 8F66798124C3CF01
Round 13   KEY: 1886D7C891E6 Text: 24C3CF017B43392E
Round 14   KEY: 75503A84CEAD Text: 7B43392E0C79C069
Round 15   KEY: A681E45A1ED1 Text: 0C79C06972406353
Round 16   KEY: 92B553E8B036 Text: F2B6777F72406353
Cipher Text: 0FDF1501D7DDEF50

DECRYPTION:
Round 1   KEY

### 3DES

In [ ]:
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]
 

exp_pbox = [32, 1, 2 , 3 , 4 , 5 , 4 , 5,
         6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
         12, 13, 12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27,
         28, 29, 28, 29, 30, 31, 32, 1 ]
         
 
straight_per = [ 16,  7, 20, 21,
        29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2,  8, 24, 14,
        32, 27,  3,  9,
        19, 13, 30,  6,
        22, 11,  4, 25 ]

sbox =  [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
          [ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
          [ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
          [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ]],
            
         [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
            [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
            [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
           [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]],
   
         [ [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
           [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
           [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ]],
       
          [ [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
           [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
           [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
            [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14] ],
        
          [ [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
           [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
            [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
           [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ]],
       
         [ [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
           [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
            [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
            [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13] ],
         
          [ [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
           [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
            [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
            [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12] ],
        
         [ [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
            [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
            [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
            [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11] ] ]
   
# Final Permutation Table
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
               39, 7, 47, 15, 55, 23, 63, 31,
               38, 6, 46, 14, 54, 22, 62, 30,
               37, 5, 45, 13, 53, 21, 61, 29,
               36, 4, 44, 12, 52, 20, 60, 28,
               35, 3, 43, 11, 51, 19, 59, 27,
               34, 2, 42, 10, 50, 18, 58, 26,
               33, 1, 41, 9, 49, 17, 57, 25 ]

keyparity = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

shift_table = [1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]
 
key_comp = [14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]


size = 64
def initialPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[initial_perm[i] - 1]
  
  return ''.join(li)

def finalPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[final_perm[i] - 1]
  
  return ''.join(li)

def expandPBox(in1):
  expanded = ''
  for i in range(len(exp_pbox)):
    expanded += str(in1[exp_pbox[i] - 1])

  return expanded

def s_box(in1, num):
  row = in1[0] + in1[5]
  col = in1[1 : 5]
  curr_sbox = sbox[num]
  row = curr_sbox[int(row, 2)]
  entry = row[int(col, 2)]
  return "{0:04b}".format(entry)

def compressBox(in1):
  res = ''
  for i in range(0, 48, 6):
    res += s_box(in1[i:i + 6], i // 6)
    # print('res', len(res), res)
  return res

def straightPBox(in1):
  li = [-1] * 32
  for i in range(len(straight_per)):
    # print(straight_per[i], len(in1))
    li[i] = in1[straight_per[i] - 1]
  
  return ''.join(li)

def roundFunction(right, key):
  in1 = expandPBox(right)
  xor_res = xor(in1, key)
  compressed = compressBox(xor_res)
  # print('---', len(compressed))
  res = straightPBox(compressed)

  return res

def round(left, right, key):
  next_l = right
  inp = roundFunction(right, key)
  next_r = xor(left, inp)

  return next_l, next_r

def xor(in1, in2):
  res = ''
  for i in range(len(in1)):
    res += str(int(in1[i]) ^ int(in2[i]))

  return res

plaintext = ''
import random
for i in range(size):
  plaintext += str(random.randint(0, 1))

def dropParity(key):
  k = ''
  for i in range(len(keyparity)):
    k += key[keyparity[i] - 1]

  return k

def shift_left(val, pos):
    s = ''
    for i in range(pos):
        for j in range(1, len(val)):
            s = s + val[j]
        s = s + val[0]
        val = s
        s = ''
    return val

def roundKeyGen(key):
  left = key[0:28]
  right = key[28:56]
  
  roundKeys  = []
  for i in range(0, 16):
      left = shift_left(left, shift_table[i])
      right = shift_left(right, shift_table[i])
      
      keyy = left + right

      round_key = ''
      for i in range(len(key_comp)):
        round_key += keyy[key_comp[i] - 1]
      
      roundKeys.append(round_key)
    
  # print('RKB\n\n', roundKeys, '\n\n\n')
  return roundKeys

def encrypt(plaintext, roundKeys):
  afterInitial = initialPermutation(plaintext)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  cipherText = finalPermutation(afterAllRounds)

  return cipherText

def decrypt(cipher, roundKeys):
  afterInitial = initialPermutation(cipher)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  plain = finalPermutation(afterAllRounds)

  return plain

def bin2hex(s):
    mp = {"0000" : '0',
          "0001" : '1',
          "0010" : '2',
          "0011" : '3',
          "0100" : '4',
          "0101" : '5',
          "0110" : '6',
          "0111" : '7',
          "1000" : '8',
          "1001" : '9',
          "1010" : 'A',
          "1011" : 'B',
          "1100" : 'C',
          "1101" : 'D',
          "1110" : 'E',
          "1111" : 'F' }
    hex = ""
    for i in range(0,len(s),4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
         
    return hex

def hex2bin(s):
    mp = {'0' : "0000",
          '1' : "0001",
          '2' : "0010",
          '3' : "0011",
          '4' : "0100",
          '5' : "0101",
          '6' : "0110",
          '7' : "0111",
          '8' : "1000",
          '9' : "1001",
          'A' : "1010",
          'B' : "1011",
          'C' : "1100",
          'D' : "1101",
          'E' : "1110",
          'F' : "1111" }
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

def threeDES_encrypt(plaintext, roundKeyss):
  # print(roundKeyss)
  p1 = encrypt(plaintext, roundKeyss[0])
  print()
  p2 = decrypt(p1, roundKeyss[1])
  print()
  p3 = encrypt(p2, roundKeyss[2])

  return p3

def threeDES_decrypt(cipher, roundKeyss):
  p1 = decrypt(cipher, roundKeyss[0][::-1])
  print()
  p2 = encrypt(p1, roundKeyss[1][::-1])
  print()
  p3 = decrypt(p2, roundKeyss[2][::-1])

  return p3


plaintext = hex2bin(input('Enter plain text in hexa-decimal format:\n'))
# plaintext = hex2bin('123456ABCD132536')
roundKeyss = []
# key1 = hex2bin('AABB09182736CCDD')
key1 = hex2bin(input('Enter key 1 in hexa-decimal format:'))
key1 = dropParity(key1)
roundKeyss.append(roundKeyGen(key1))
# key1 = hex2bin('AEBB00182736ECDD')
key1 = hex2bin(input('Enter key 2 in hexa-decimal format:'))
key1 = dropParity(key1)
roundKeyss.append(roundKeyGen(key1))
key1 = hex2bin(input('Enter key 3 in hexa-decimal format:'))
# key1 = hex2bin('AAB209282336DCDD')
key1 = dropParity(key1)
roundKeyss.append(roundKeyGen(key1))

print('\nENCRYPTION:')
cipher = threeDES_encrypt(plaintext, roundKeyss)
print('Cipher Text:',bin2hex(cipher))
print('\nDECRYPTION:')
plain = threeDES_decrypt(cipher, roundKeyss[::-1])
print('Plain Text:', bin2hex(plain))

Enter plain text in hexa-decimal format:
ABCD1234567891DC
Enter key 1 in hexa-decimal format:1234567891ABCD56
Enter key 2 in hexa-decimal format:ABCD123456789764
Enter key 3 in hexa-decimal format:ABCD1234543289654

ENCRYPTION:
Round 1   KEY: 1A024FD67174 Text: C329A315F7F2CACE
Round 2   KEY: 692ED1CDC8C7 Text: F7F2CACE20D5030F
Round 3   KEY: 55F48846E6D9 Text: 20D5030FBD8F5A48
Round 4   KEY: 5285D2BB954D Text: BD8F5A48BC45A4ED
Round 5   KEY: 7CC8078AD7A2 Text: BC45A4ED0C21F392
Round 6   KEY: 23A10E5C6F25 Text: 0C21F392DEDD2FF0
Round 7   KEY: 2804B7FA48D8 Text: DEDD2FF069ABF987
Round 8   KEY: F50838C1F31B Text: 69ABF987E8D66D8B
Round 9   KEY: 40D9EAAD8F99 Text: E8D66D8BD8CB9518
Round 10   KEY: B0E1471B5657 Text: D8CB95187B4AA78F
Round 11   KEY: 214F035FC1A4 Text: 7B4AA78F2FF08B81
Round 12   KEY: 6131B5806DCD Text: 2FF08B813D7F6167
Round 13   KEY: 9584D1EAB295 Text: 3D7F616754EF2A74
Round 14   KEY: 5742B2F347AB Text: 54EF2A74C7B41CC3
Round 15   KEY: BE91841E1B0B Text: C7B41CC38811757A
R

# Week 5

### AES

In [ ]:
state = [['32', '88', '31', 'e0'],
['43', '5a', '31', '37'],
['f6', '30', '98', '07'],
['a8', '8d', 'a2', '34']]

key = [['2b', '28', 'ab', '09'],
['7e', 'ae', 'f7', 'cf'],
['15', 'd2', '15', '4f'],
['16', 'a6', '88', '3c']]

def multiply_ints_as_polynomials(x, y):
	z = 0
	while x != 0:
		if x & 1 == 1:
			z ^= y
		y <<= 1
		x >>= 1
	return z

def number_bits(x):
	nb = 0
	while x != 0:
		nb += 1
		x >>= 1
	return nb

def mod_int_as_polynomial(x, m):
	nbm = number_bits(m)
	while True:
		nbx = number_bits(x) 
		if nbx < nbm:
			return x
		mshift = m << (nbx - nbm)
		x ^= mshift

def rijndael_multiplication(x, y):
	z = multiply_ints_as_polynomials(x, y)
	m = int('100011011', 2)
	return mod_int_as_polynomial(z, m)

def rijndael_inverse(x):
	if x == 0:
		return 0
	for y in range(1, 256):
		if rijndael_multiplication(x, y) == 1:
			return y

def dot_product(x, y):
	z = x & y
	dot = 0  
	while z != 0:
		dot ^= z & 1
		z >>= 1
	return dot 

def affine_transformation(A, x, b):
	y = 0
	for i in reversed(range(8)):
		row = (A >> 8 * i) & 0xff 
		bit = dot_product(row, x)
		y ^= (bit << i)
	return y ^ b

def rijndael_sbox(x):
	xinv = rijndael_inverse(x)
	A = int('1111100001111100001111100001111110001111110001111110001111110001', 2)
	b = int('01100011', 2)
	return affine_transformation(A, xinv, b)


def get_rijndael_sbox():
  s_box = []
  for row in range(16):
	  li = []
	  for col in range(16):
	    x = 16 * row + col
	    hexstring = format(rijndael_sbox(x), "02x")
	    li.append(hexstring)
	  s_box.append(li)
  return s_box

hex2int = {}
for i in range(0, 10):
  hex2int[str(i)] = i
hex2int['a'] = 10
hex2int['b'] = 11
hex2int['c'] = 12
hex2int['d'] = 13
hex2int['e'] = 14
hex2int['f'] = 15

int2hex = {}
for i in range(0, 10):
  int2hex[i] = str(i)
int2hex[10] = 'a'
int2hex[11] = 'b'
int2hex[12] = 'c'
int2hex[13] = 'd'
int2hex[14] = 'e'
int2hex[15] = 'f'

def get_rijndael_inverse_sbox(s_box):
  inverse_sBox = []
  for i in range(16):
    inverse_sBox.append([-1] * 16)
  for i in range(16):
    for j in range(16):
      inverse_sBox[hex2int[s_box[i][j][0]]][hex2int[s_box[i][j][1]]] = int2hex[i] + int2hex[j]

  return inverse_sBox

s_box = get_rijndael_sbox()

inverse_sBox = get_rijndael_inverse_sbox(s_box)

def bin2hex(s):
    mp = {"0000" : '0',
          "0001" : '1',
          "0010" : '2',
          "0011" : '3',
          "0100" : '4',
          "0101" : '5',
          "0110" : '6',
          "0111" : '7',
          "1000" : '8',
          "1001" : '9',
          "1010" : 'a',
          "1011" : 'b',
          "1100" : 'c',
          "1101" : 'd',
          "1110" : 'e',
          "1111" : 'f' }
    hex = ""
    for i in range(0,len(s),4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
         
    return hex

def hex2bin(s):
    mp = {'0' : "0000",
          '1' : "0001",
          '2' : "0010",
          '3' : "0011",
          '4' : "0100",
          '5' : "0101",
          '6' : "0110",
          '7' : "0111",
          '8' : "1000",
          '9' : "1001",
          'a' : "1010",
          'b' : "1011",
          'c' : "1100",
          'd' : "1101",
          'e' : "1110",
          'f' : "1111" }
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

def xor(a, b):
  res = ''
  in1 = hex2bin(a)
  in2 = hex2bin(b)
  for i in range(len(in1)):
    res += str(int(in1[i]) ^ int(in2[i]))
  inHex = bin2hex(res[:4]) + bin2hex(res[4:])

  return inHex

def addRoundKey(state, key):
  newState = []
  for i in range(4):
    li = []
    for j in range(4):
      res1 = xor(state[i][j], key[i][j])
      li.append(res1)
    newState.append(li)
  return newState

def initialRound(state, key):
  return addRoundKey(state, key)

def initialDec(state, key):
  return addRoundKey(state, key)

def subBytes(state):
  newState = []
  for i in range(4):
    li = []
    for j in range(4):
      row = hex2int[state[i][j][0]]
      col = hex2int[state[i][j][1]]
      li.append(s_box[row][col])
    newState.append(li)
  return newState

def subBytesInverse(state):
  newState = []
  for i in range(4):
    li = []
    for j in range(4):
      row = hex2int[state[i][j][0]]
      col = hex2int[state[i][j][1]]
      li.append(inverse_sBox[row][col])
    newState.append(li)
  return newState

def shiftRows(s):
  s[1][0], s[1][1], s[1][2], s[1][3] = s[1][1], s[1][2], s[1][3], s[1][0]
  s[2][0], s[2][1], s[2][2], s[2][3] = s[2][2], s[2][3], s[2][0], s[2][1]
  s[3][0], s[3][1], s[3][2], s[3][3] = s[3][3], s[3][0], s[3][1], s[3][2]

  return s

def shiftRowsRev(s):
  s[1][0], s[1][1], s[1][2], s[1][3] = s[1][3], s[1][0], s[1][1], s[1][2]
  s[2][0], s[2][1], s[2][2], s[2][3] = s[2][2], s[2][3], s[2][0], s[2][1]
  s[3][0], s[3][1], s[3][2], s[3][3] = s[3][1], s[3][2], s[3][3], s[3][0]

  return s

def multiply(a, b):
  if (a == '01'):
    return b
  binn = hex2bin(b[0]) + hex2bin(b[1])
  shifted = '{:08b}'.format((int(binn, 2) << 1))
  if (len(shifted) > 8):
    shifted = shifted[1:]
  if (a == '02'):
    if (binn[0] == '0'):
      return bin2hex(shifted)
    else:
      res = xor(bin2hex(shifted), bin2hex('00011011'))
      return res
  else:
    if (binn[0] == '0'):
      res = xor(bin2hex(shifted), bin2hex(binn))
      return res
    else:
      res = xor(bin2hex(shifted), bin2hex('00011011'))
      res = xor(res, bin2hex(binn))
      return res


def solve(state, C, i, j):
  res = '00'
  for k in range(4):
    t = multiply(C[i][k], state[k][j])
    res = xor(res, t)
  return res
    

def mixCols(state):
  C = [['02', '03', '01', '01'],
  ['01', '02', '03', '01'],
  ['01', '01', '02', '03'],
  ['03', '01', '01', '02']]

  newState = []
  for i in range(4):
    newState.append([-1] * 4)
    for j in range(4):
      newState[i][j] = solve(state, C, i, j)
  return newState

def mixColsDec(state):
  C = [['0e', '0b', '0d', '09'],
  ['09', '0e', '0b', '0d'],
  ['0d', '09', '0e', '0b'],
  ['0b', '0d', '09', '0e']]

  newState = []
  for i in range(4):
    newState.append([-1] * 4)
    for j in range(4):
      newState[i][j] = solve(state, C, i, j)
  return newState


def eachRound(state, key):
  state = subBytes(state)
  state = shiftRows(state)
  state = mixCols(state)
  state = addRoundKey(state, key)
  return state

def eachRoundDec(state, key):
  state = addRoundKey(state, key)
  state = mixColsDec(state)
  state = shiftRowsRev(state)
  state = subBytesInverse(state)
  return state

def finalRound(state, key):
  state = subBytes(state)
  state = shiftRows(state)
  state = addRoundKey(state, key)
  return state

def finalDec(cipher, key):
  state = addRoundKey(cipher, key)
  state = shiftRowsRev(state)
  state = subBytesInverse(state)
  return state

rcon = ['01', '02', '04', '08', '10', '20', '40', '80', '1b', '36']
def generateKeys(key):
  keys = []
  for roundd in range(10):
    li = [key[1][3], key[2][3], key[3][3], key[0][3]]
    for i in range(4):
      li[i] = s_box[hex2int[li[i][0]]][hex2int[li[i][1]]]
    rconn = [rcon[roundd], '00', '00', '00']
    keyNew = []
    for i in range(4):
      keyNew.append([-1] * 4)
    for i in range(4):
      keyNew[i][0] = xor(xor(li[i], rconn[i]), key[i][0])
    for i in range(1, 4):
      for j in range(4):
        keyNew[j][i] = xor(keyNew[j][i - 1], key[j][i])
    keys.append(keyNew)
  return keys

roundKeyss = generateKeys(key)
def updateKey(key, r):
  return roundKeyss[r]

def encrypt(state, key):
  state = initialRound(state, key)
  print('After initial round:', state)
  key = updateKey(key, 0)
  for i in range(9):
    state = eachRound(state, key)
    key = updateKey(key, i + 1)
    print('Plain text after round', (i + 1), ':', state)
    print('Round Key', (i + 1), ':', key, '\n')
  state = finalRound(state, key)
  print('Cipher text:', state)
  return state

def decrypt(cipher, key):
  key = updateKey(key, 9)
  state = finalDec(cipher, key)
  for i in range(8, -1, -1):
    state = eachRoundDec(state, key)
    key = updateKey(key, i + 1)

  state = initialDec(state, key)

  return state

keyyy = key
cipher = encrypt(state, key)
plain = decrypt(cipher, keyyy)
print('Plain text:', plain)

After initial round: [['19', 'a0', '9a', 'e9'], ['3d', 'f4', 'c6', 'f8'], ['e3', 'e2', '8d', '48'], ['be', '2b', '2a', '08']]
Plain text after round 1 : [['a4', '68', '6b', '02'], ['9c', '9f', '5b', '6a'], ['7f', '35', 'ea', '50'], ['f2', '2b', '43', '49']]
Round Key 1 : [['a3', '8b', '20', '29'], ['fa', '54', 'a3', '6c'], ['fe', '2c', '39', '76'], ['17', 'b1', '39', '05']] 

Plain text after round 2 : [['fb', '90', 'fb', '32'], ['b7', '1f', '44', '07'], ['34', '76', 'f3', 'c6'], ['e6', '1d', '91', 'e0']]
Round Key 2 : [['a5', '8d', '26', '2f'], ['fa', '54', 'a3', '6c'], ['fe', '2c', '39', '76'], ['17', 'b1', '39', '05']] 

Plain text after round 3 : [['0c', '5a', 'd0', '30'], ['98', '4b', 'b1', 'cf'], ['13', 'ad', '34', '14'], ['12', 'b9', 'a6', 'e8']]
Round Key 3 : [['a9', '81', '2a', '23'], ['fa', '54', 'a3', '6c'], ['fe', '2c', '39', '76'], ['17', 'b1', '39', '05']] 

Plain text after round 4 : [['03', '96', '64', '50'], ['ca', 'bd', '0d', '64'], ['35', 'f5', 'c3', '69'], ['88', 'd

### AES S-box

In [ ]:
def multiply_ints_as_polynomials(x, y):
	z = 0
	while x != 0:
		if x & 1 == 1:
			z ^= y
		y <<= 1
		x >>= 1
	return z

def number_bits(x):
	nb = 0
	while x != 0:
		nb += 1
		x >>= 1
	return nb

def mod_int_as_polynomial(x, m):
	nbm = number_bits(m)
	while True:
		nbx = number_bits(x) 
		if nbx < nbm:
			return x
		mshift = m << (nbx - nbm)
		x ^= mshift

def rijndael_multiplication(x, y):
	z = multiply_ints_as_polynomials(x, y)
	m = int('100011011', 2)
	return mod_int_as_polynomial(z, m)

def rijndael_inverse(x):
	if x == 0:
		return 0
	for y in range(1, 256):
		if rijndael_multiplication(x, y) == 1:
			return y

def dot_product(x, y):
	z = x & y
	dot = 0  
	while z != 0:
		dot ^= z & 1
		z >>= 1
	return dot 

def affine_transformation(A, x, b):
	y = 0
	for i in reversed(range(8)):
		row = (A >> 8 * i) & 0xff 
		bit = dot_product(row, x)
		y ^= (bit << i)
	return y ^ b

def rijndael_sbox(x):
	xinv = rijndael_inverse(x)
	A = int('1111100001111100001111100001111110001111110001111110001111110001', 2)
	b = int('01100011', 2)
	return affine_transformation(A, xinv, b)


def get_rijndael_sbox():
  s_box = []
  for row in range(16):
	  li = []
	  for col in range(16):
	    x = 16 * row + col
	    hexstring = format(rijndael_sbox(x), "02x")
	    li.append(hexstring)
	  s_box.append(li)
  return s_box

hex2int = {}
for i in range(0, 10):
  hex2int[str(i)] = i
hex2int['a'] = 10
hex2int['b'] = 11
hex2int['c'] = 12
hex2int['d'] = 13
hex2int['e'] = 14
hex2int['f'] = 15

int2hex = {}
for i in range(0, 10):
  int2hex[i] = str(i)
int2hex[10] = 'a'
int2hex[11] = 'b'
int2hex[12] = 'c'
int2hex[13] = 'd'
int2hex[14] = 'e'
int2hex[15] = 'f'

def get_rijndael_inverse_sbox(s_box):
  inverse_sBox = []
  for i in range(16):
    inverse_sBox.append([-1] * 16)
  for i in range(16):
    for j in range(16):
      inverse_sBox[hex2int[s_box[i][j][0]]][hex2int[s_box[i][j][1]]] = int2hex[i] + int2hex[j]

  return inverse_sBox

s_box = get_rijndael_sbox()
print("S BOX\n", s_box)

inverse_sBox = get_rijndael_inverse_sbox(s_box)
print('Inverse S BOX\n', inverse_sBox)

S BOX
 [['63', '7c', '77', '7b', 'f2', '6b', '6f', 'c5', '30', '01', '67', '2b', 'fe', 'd7', 'ab', '76'], ['ca', '82', 'c9', '7d', 'fa', '59', '47', 'f0', 'ad', 'd4', 'a2', 'af', '9c', 'a4', '72', 'c0'], ['b7', 'fd', '93', '26', '36', '3f', 'f7', 'cc', '34', 'a5', 'e5', 'f1', '71', 'd8', '31', '15'], ['04', 'c7', '23', 'c3', '18', '96', '05', '9a', '07', '12', '80', 'e2', 'eb', '27', 'b2', '75'], ['09', '83', '2c', '1a', '1b', '6e', '5a', 'a0', '52', '3b', 'd6', 'b3', '29', 'e3', '2f', '84'], ['53', 'd1', '00', 'ed', '20', 'fc', 'b1', '5b', '6a', 'cb', 'be', '39', '4a', '4c', '58', 'cf'], ['d0', 'ef', 'aa', 'fb', '43', '4d', '33', '85', '45', 'f9', '02', '7f', '50', '3c', '9f', 'a8'], ['51', 'a3', '40', '8f', '92', '9d', '38', 'f5', 'bc', 'b6', 'da', '21', '10', 'ff', 'f3', 'd2'], ['cd', '0c', '13', 'ec', '5f', '97', '44', '17', 'c4', 'a7', '7e', '3d', '64', '5d', '19', '73'], ['60', '81', '4f', 'dc', '22', '2a', '90', '88', '46', 'ee', 'b8', '14', 'de', '5e', '0b', 'db'], ['e0', '32',

# Week 6

### AES using CBC mode

In [ ]:
def number_bits(x):
	nb = 0
	while x != 0:
		nb += 1
		x >>= 1
	return nb

def getState(text):
    matrix = []
    for i in range(16):

        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[i // 4].append(byte)

    return matrix

def multiply_ints_as_polynomials(x, y):
	z = 0
	while x != 0:
		if x & 1 == 1:
			z ^= y
		y <<= 1
		x >>= 1
	return z

def blockToHex(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))

    return text

def mod_int_as_polynomial(x, m):
	nbm = number_bits(m)
	while True:
		nbx = number_bits(x) 
		if nbx < nbm:
			return x
		mshift = m << (nbx - nbm)
		x ^= mshift

def rijndael_multiplication(x, y):
	z = multiply_ints_as_polynomials(x, y)
	m = int('100011011', 2)
	return mod_int_as_polynomial(z, m)

def rijndael_inverse(x):
	if x == 0:
		return 0
	for y in range(1, 256):
		if rijndael_multiplication(x, y) == 1:
			return y

def dot_product(x, y):
	z = x & y
	dot = 0  
	while z != 0:
		dot ^= z & 1
		z >>= 1
	return dot 

def affine_transformation(A, x, b):
	y = 0
	for i in reversed(range(8)):
		row = (A >> 8 * i) & 0xff 
		bit = dot_product(row, x)
		y ^= (bit << i)
	return y ^ b

def rijndael_sbox(x):
	xinv = rijndael_inverse(x)
	A = int('1111100001111100001111100001111110001111110001111110001111110001', 2)
	b = int('01100011', 2)
	return affine_transformation(A, xinv, b)


def get_rijndael_sbox():
  s_box = []
  for row in range(16):
	  li = []
	  for col in range(16):
	    x = 16 * row + col
	    hexstring = format(rijndael_sbox(x), "02x")
	    li.append(hexstring)
	  s_box.append(li)
  return s_box

hex2int = {}
for i in range(0, 10):
  hex2int[str(i)] = i
hex2int['a'] = 10
hex2int['b'] = 11
hex2int['c'] = 12
hex2int['d'] = 13
hex2int['e'] = 14
hex2int['f'] = 15

int2hex = {}
for i in range(0, 10):
  int2hex[i] = str(i)
int2hex[10] = 'a'
int2hex[11] = 'b'
int2hex[12] = 'c'
int2hex[13] = 'd'
int2hex[14] = 'e'
int2hex[15] = 'f'

def get_rijndael_inverse_sbox(s_box):
  inverse_sBox = []
  for i in range(16):
    inverse_sBox.append([-1] * 16)
  for i in range(16):
    for j in range(16):
      inverse_sBox[hex2int[s_box[i][j][0]]][hex2int[s_box[i][j][1]]] = int2hex[i] + int2hex[j]

  return inverse_sBox

Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

# Inv S-box
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

# exponentiation of 2
Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

class AES:
    def __init__(self, key):
        self.keysUtil(key)

    def keysUtil(self, key):
        self.roundKeys = getState(key)
        for i in range(4, 4 * 11):
            self.roundKeys.append([])
            if i % 4 == 0:
                byte = self.roundKeys[i - 4][0] ^ Sbox[self.roundKeys[i - 1][1]] ^ Rcon[i // 4]
                self.roundKeys[i].append(byte)

                for j in range(1, 4):
                    byte = self.roundKeys[i - 4][j] ^ Sbox[self.roundKeys[i - 1][(j + 1) % 4]]
                    self.roundKeys[i].append(byte)
            else:
                for j in range(4):
                    byte = self.roundKeys[i - 4][j] ^ self.roundKeys[i - 1][j]
                    self.roundKeys[i].append(byte)

    def addRoundKey(self, s, k):
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]

    def subBytes(self, s):
        for i in range(4):
            for j in range(4):
                s[i][j] = Sbox[s[i][j]]

    def subBytesInv(self, s):
        for i in range(4):
            for j in range(4):
                s[i][j] = InvSbox[s[i][j]]

    def leftShiftRows(self, s):
        s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

    def rightShiftRows(self, s):
        s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]

    def mixC(self, a):
        t = a[0] ^ a[1] ^ a[2] ^ a[3]
        u = a[0]
        a[0] ^= t ^ xtime(a[0] ^ a[1])
        a[1] ^= t ^ xtime(a[1] ^ a[2])
        a[2] ^= t ^ xtime(a[2] ^ a[3])
        a[3] ^= t ^ xtime(a[3] ^ u)

    def mixCols(self, s):
        for i in range(4):
            self.mixC(s[i])

    def mixColsInv(self, s):
        
        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        self.mixCols(s)

    def eachRoundEnc(self, state_matrix, key_matrix):
        self.subBytes(state_matrix)
        self.leftShiftRows(state_matrix)
        self.mixCols(state_matrix)
        self.addRoundKey(state_matrix, key_matrix)

    def eachRoundDec(self, state_matrix, key_matrix):
        self.addRoundKey(state_matrix, key_matrix)
        self.mixColsInv(state_matrix)
        self.rightShiftRows(state_matrix)
        self.subBytesInv(state_matrix)

    def encrypt(self, plaintext):
        self.plain_state = getState(plaintext)
        self.addRoundKey(self.plain_state, self.roundKeys[:4])

        for i in range(1, 10):
            self.eachRoundEnc(self.plain_state, self.roundKeys[4 * i : 4 * (i + 1)])
        
        self.subBytes(self.plain_state)
        self.leftShiftRows(self.plain_state)
        self.addRoundKey(self.plain_state, self.roundKeys[40:])
        return blockToHex(self.plain_state)

    def decrypt(self, ciphertext):
        self.cipher_state = getState(ciphertext)
        self.addRoundKey(self.cipher_state, self.roundKeys[40:])
        self.rightShiftRows(self.cipher_state)
        self.subBytesInv(self.cipher_state)

        for i in range(9, 0, -1):
            self.eachRoundDec(self.cipher_state, self.roundKeys[4 * i : 4 * (i + 1)])
        self.addRoundKey(self.cipher_state, self.roundKeys[:4])
        return blockToHex(self.cipher_state)

IV = 0x127689abcd4345ba127689abcd4345ba
plaintext = [0xa308d31313243f6a888598a2e0370734, 0x13198a2e0373243f6a8885a308d30734]
print('PLAIN TEXT:')
for i in plaintext:
  print('{:x}'.format(i), end='')
print()

key = 0x28aed2a6abf7158802b7e15169cf4f3c
print('KEY:', '{:x}'.format(key))

aes = AES(key)

ciphertext = []
param1 = IV
for text in plaintext:
  xorr = param1 ^ text
  cipher = aes.encrypt(xorr)
  ciphertext.append(cipher)
  param1 = cipher

plaintextDec = []
param1 = IV
for cipher in ciphertext:
  plainn = aes.decrypt(cipher)
  plaintextDec.append(plainn ^ param1)
  param1 = cipher

print('Encrypted:')
for i in ciphertext:
  print('{:x}'.format(i), end='')

print('\nDecrypted:')
for i in plaintextDec:
  print('{:x}'.format(i), end='')

PLAIN TEXT:
a308d31313243f6a888598a2e037073413198a2e0373243f6a8885a308d30734
KEY: 28aed2a6abf7158802b7e15169cf4f3c
Encrypted:
312d433da38f2a8e464fd2404cee605f5722d8cf90543f18f26101e92b2309c5
Decrypted:
a308d31313243f6a888598a2e037073413198a2e0373243f6a8885a308d30734

### AES using Counter mode

In [ ]:
def number_bits(x):
	nb = 0
	while x != 0:
		nb += 1
		x >>= 1
	return nb

def getState(text):
    matrix = []
    for i in range(16):

        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[i // 4].append(byte)

    return matrix

def multiply_ints_as_polynomials(x, y):
	z = 0
	while x != 0:
		if x & 1 == 1:
			z ^= y
		y <<= 1
		x >>= 1
	return z

def blockToHex(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))

    return text

def mod_int_as_polynomial(x, m):
	nbm = number_bits(m)
	while True:
		nbx = number_bits(x) 
		if nbx < nbm:
			return x
		mshift = m << (nbx - nbm)
		x ^= mshift

def rijndael_multiplication(x, y):
	z = multiply_ints_as_polynomials(x, y)
	m = int('100011011', 2)
	return mod_int_as_polynomial(z, m)

def rijndael_inverse(x):
	if x == 0:
		return 0
	for y in range(1, 256):
		if rijndael_multiplication(x, y) == 1:
			return y

def dot_product(x, y):
	z = x & y
	dot = 0  
	while z != 0:
		dot ^= z & 1
		z >>= 1
	return dot 

def affine_transformation(A, x, b):
	y = 0
	for i in reversed(range(8)):
		row = (A >> 8 * i) & 0xff 
		bit = dot_product(row, x)
		y ^= (bit << i)
	return y ^ b

def rijndael_sbox(x):
	xinv = rijndael_inverse(x)
	A = int('1111100001111100001111100001111110001111110001111110001111110001', 2)
	b = int('01100011', 2)
	return affine_transformation(A, xinv, b)


def get_rijndael_sbox():
  s_box = []
  for row in range(16):
	  li = []
	  for col in range(16):
	    x = 16 * row + col
	    hexstring = format(rijndael_sbox(x), "02x")
	    li.append(hexstring)
	  s_box.append(li)
  return s_box

hex2int = {}
for i in range(0, 10):
  hex2int[str(i)] = i
hex2int['a'] = 10
hex2int['b'] = 11
hex2int['c'] = 12
hex2int['d'] = 13
hex2int['e'] = 14
hex2int['f'] = 15

int2hex = {}
for i in range(0, 10):
  int2hex[i] = str(i)
int2hex[10] = 'a'
int2hex[11] = 'b'
int2hex[12] = 'c'
int2hex[13] = 'd'
int2hex[14] = 'e'
int2hex[15] = 'f'

def get_rijndael_inverse_sbox(s_box):
  inverse_sBox = []
  for i in range(16):
    inverse_sBox.append([-1] * 16)
  for i in range(16):
    for j in range(16):
      inverse_sBox[hex2int[s_box[i][j][0]]][hex2int[s_box[i][j][1]]] = int2hex[i] + int2hex[j]

  return inverse_sBox

Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

# Inv S-box
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

# exponentiation of 2
Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)

class AES:
    def __init__(self, key):
        self.keysUtil(key)

    def keysUtil(self, key):
        self.roundKeys = getState(key)
        for i in range(4, 4 * 11):
            self.roundKeys.append([])
            if i % 4 == 0:
                byte = self.roundKeys[i - 4][0] ^ Sbox[self.roundKeys[i - 1][1]] ^ Rcon[i // 4]
                self.roundKeys[i].append(byte)

                for j in range(1, 4):
                    byte = self.roundKeys[i - 4][j] ^ Sbox[self.roundKeys[i - 1][(j + 1) % 4]]
                    self.roundKeys[i].append(byte)
            else:
                for j in range(4):
                    byte = self.roundKeys[i - 4][j] ^ self.roundKeys[i - 1][j]
                    self.roundKeys[i].append(byte)

    def addRoundKey(self, s, k):
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]

    def subBytes(self, s):
        for i in range(4):
            for j in range(4):
                s[i][j] = Sbox[s[i][j]]

    def subBytesInv(self, s):
        for i in range(4):
            for j in range(4):
                s[i][j] = InvSbox[s[i][j]]

    def leftShiftRows(self, s):
        s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

    def rightShiftRows(self, s):
        s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
        s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
        s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]

    def mixC(self, a):
        t = a[0] ^ a[1] ^ a[2] ^ a[3]
        u = a[0]
        a[0] ^= t ^ xtime(a[0] ^ a[1])
        a[1] ^= t ^ xtime(a[1] ^ a[2])
        a[2] ^= t ^ xtime(a[2] ^ a[3])
        a[3] ^= t ^ xtime(a[3] ^ u)

    def mixCols(self, s):
        for i in range(4):
            self.mixC(s[i])

    def mixColsInv(self, s):
        
        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        self.mixCols(s)

    def eachRoundEnc(self, state_matrix, key_matrix):
        self.subBytes(state_matrix)
        self.leftShiftRows(state_matrix)
        self.mixCols(state_matrix)
        self.addRoundKey(state_matrix, key_matrix)

    def eachRoundDec(self, state_matrix, key_matrix):
        self.addRoundKey(state_matrix, key_matrix)
        self.mixColsInv(state_matrix)
        self.rightShiftRows(state_matrix)
        self.subBytesInv(state_matrix)

    def encrypt(self, plaintext):
        self.plain_state = getState(plaintext)
        self.addRoundKey(self.plain_state, self.roundKeys[:4])

        for i in range(1, 10):
            self.eachRoundEnc(self.plain_state, self.roundKeys[4 * i : 4 * (i + 1)])
        
        self.subBytes(self.plain_state)
        self.leftShiftRows(self.plain_state)
        self.addRoundKey(self.plain_state, self.roundKeys[40:])
        return blockToHex(self.plain_state)

    def decrypt(self, ciphertext):
        self.cipher_state = getState(ciphertext)
        self.addRoundKey(self.cipher_state, self.roundKeys[40:])
        self.rightShiftRows(self.cipher_state)
        self.subBytesInv(self.cipher_state)

        for i in range(9, 0, -1):
            self.eachRoundDec(self.cipher_state, self.roundKeys[4 * i : 4 * (i + 1)])
        self.addRoundKey(self.cipher_state, self.roundKeys[:4])
        return blockToHex(self.cipher_state)

counter = [0x00000000000000000000000000000001, 0x00000000000000000000000000000002]
plaintext = [0xa308d31313243f6a888598a2e0370734, 0x13198a2e0373243f6a8885a308d30734]
print('PLAIN TEXT:')
for i in plaintext:
  print('{:x}'.format(i), end='')
print()

key = 0x28aed2a6abf7158802b7e15169cf4f3c
print('KEY:', '{:x}'.format(key))

aes = AES(key)

ciphertext = []
i = 0
for text in plaintext:
  cip = aes.encrypt(counter[i])
  xorr = cip ^ text
  ciphertext.append(xorr)
  i += 1
i = 0
plaintextDec = []
for cipher in ciphertext:
  plainn = aes.encrypt(counter[i])
  xorr = plainn ^ cipher
  plaintextDec.append(xorr)
  i += 1

print('Encrypted:')
for i in ciphertext:
  print('{:x}'.format(i), end='')

print('\nDecrypted:')
for i in plaintextDec:
  print('{:x}'.format(i), end='')

PLAIN TEXT:
a308d31313243f6a888598a2e037073413198a2e0373243f6a8885a308d30734
KEY: 28aed2a6abf7158802b7e15169cf4f3c
Encrypted:
3c1a4a44ca818b5daa550a722c0e040dff994653265cc05009025a7aea9f22b2
Decrypted:
a308d31313243f6a888598a2e037073413198a2e0373243f6a8885a308d30734

### DES using ECB

In [ ]:
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]
 

exp_pbox = [32, 1, 2 , 3 , 4 , 5 , 4 , 5,
         6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
         12, 13, 12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27,
         28, 29, 28, 29, 30, 31, 32, 1 ]
         
 
straight_per = [ 16,  7, 20, 21,
        29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2,  8, 24, 14,
        32, 27,  3,  9,
        19, 13, 30,  6,
        22, 11,  4, 25 ]

sbox =  [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
          [ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
          [ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
          [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ]],
            
         [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
            [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
            [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
           [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]],
   
         [ [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
           [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
           [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ]],
       
          [ [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
           [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
           [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
            [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14] ],
        
          [ [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
           [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
            [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
           [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ]],
       
         [ [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
           [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
            [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
            [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13] ],
         
          [ [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
           [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
            [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
            [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12] ],
        
         [ [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
            [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
            [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
            [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11] ] ]
   
# Final Permutation Table
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
               39, 7, 47, 15, 55, 23, 63, 31,
               38, 6, 46, 14, 54, 22, 62, 30,
               37, 5, 45, 13, 53, 21, 61, 29,
               36, 4, 44, 12, 52, 20, 60, 28,
               35, 3, 43, 11, 51, 19, 59, 27,
               34, 2, 42, 10, 50, 18, 58, 26,
               33, 1, 41, 9, 49, 17, 57, 25 ]

keyparity = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

shift_table = [1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]
 
key_comp = [14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]


size = 64
def initialPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[initial_perm[i] - 1]
  
  return ''.join(li)

def finalPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[final_perm[i] - 1]
  
  return ''.join(li)

def expandPBox(in1):
  expanded = ''
  for i in range(len(exp_pbox)):
    expanded += str(in1[exp_pbox[i] - 1])

  return expanded

def s_box(in1, num):
  row = in1[0] + in1[5]
  col = in1[1 : 5]
  curr_sbox = sbox[num]
  row = curr_sbox[int(row, 2)]
  entry = row[int(col, 2)]
  return "{0:04b}".format(entry)

def compressBox(in1):
  res = ''
  for i in range(0, 48, 6):
    res += s_box(in1[i:i + 6], i // 6)
    # print('res', len(res), res)
  return res

def straightPBox(in1):
  li = [-1] * 32
  for i in range(len(straight_per)):
    # print(straight_per[i], len(in1))
    li[i] = in1[straight_per[i] - 1]
  
  return ''.join(li)

def roundFunction(right, key):
  in1 = expandPBox(right)
  xor_res = xor(in1, key)
  compressed = compressBox(xor_res)
  # print('---', len(compressed))
  res = straightPBox(compressed)

  return res

def round(left, right, key):
  next_l = right
  inp = roundFunction(right, key)
  next_r = xor(left, inp)

  return next_l, next_r

def xor(in1, in2):
  res = ''
  for i in range(len(in1)):
    res += str(int(in1[i]) ^ int(in2[i]))

  return res

plaintext = ''
import random
for i in range(size):
  plaintext += str(random.randint(0, 1))

def dropParity(key):
  k = ''
  for i in range(len(keyparity)):
    k += key[keyparity[i] - 1]

  return k

def shift_left(val, pos):
    s = ''
    for i in range(pos):
        for j in range(1, len(val)):
            s = s + val[j]
        s = s + val[0]
        val = s
        s = ''
    return val

def roundKeyGen(key):
  left = key[0:28]
  right = key[28:56]
  
  roundKeys  = []
  for i in range(0, 16):
      left = shift_left(left, shift_table[i])
      right = shift_left(right, shift_table[i])
      
      keyy = left + right

      round_key = ''
      for i in range(len(key_comp)):
        round_key += keyy[key_comp[i] - 1]
      
      roundKeys.append(round_key)
    
  # print('RKB\n\n', roundKeys, '\n\n\n')
  return roundKeys

def encrypt(plaintext, roundKeys):
  afterInitial = initialPermutation(plaintext)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    # print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  cipherText = finalPermutation(afterAllRounds)

  return cipherText

def decrypt(cipher, roundKeys):
  afterInitial = initialPermutation(cipher)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    # print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  plain = finalPermutation(afterAllRounds)

  return plain

def bin2hex(s):
    mp = {"0000" : '0',
          "0001" : '1',
          "0010" : '2',
          "0011" : '3',
          "0100" : '4',
          "0101" : '5',
          "0110" : '6',
          "0111" : '7',
          "1000" : '8',
          "1001" : '9',
          "1010" : 'A',
          "1011" : 'B',
          "1100" : 'C',
          "1101" : 'D',
          "1110" : 'E',
          "1111" : 'F' }
    hex = ""
    for i in range(0,len(s),4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
         
    return hex

def hex2bin(s):
    mp = {'0' : "0000",
          '1' : "0001",
          '2' : "0010",
          '3' : "0011",
          '4' : "0100",
          '5' : "0101",
          '6' : "0110",
          '7' : "0111",
          '8' : "1000",
          '9' : "1001",
          'A' : "1010",
          'B' : "1011",
          'C' : "1100",
          'D' : "1101",
          'E' : "1110",
          'F' : "1111" }
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

plaintext = input('Enter plain text in hexa-decimal format:\n')
key = hex2bin(input('Enter key in hexa-decimal format:'))
key = dropParity(key)
roundKeys = roundKeyGen(key)
cipher = []
plain = []
nn = len(plaintext)
while (len(plaintext) % 16 != 0):
  plaintext += '1'

for i in range(0, len(plaintext), 16):
  cipher1 = encrypt(hex2bin(plaintext[i : i + 16]), roundKeys)
  plain1 = decrypt(cipher1, roundKeys[::-1])
  cipher.append(bin2hex(cipher1))
  plain.append(bin2hex(plain1))

print('Encrypted:', ''.join(cipher))
print('Decrypted:', ''.join(plain)[:nn])

Enter plain text in hexa-decimal format:
ABCD678965432156
Enter key in hexa-decimal format:8765432908746ABC
Encrypted: 3A4D520BA9931673
Decrypted: ABCD678965432156


### DES using CFB

In [ ]:
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]
 

exp_pbox = [32, 1, 2 , 3 , 4 , 5 , 4 , 5,
         6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
         12, 13, 12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27,
         28, 29, 28, 29, 30, 31, 32, 1 ]
         
 
straight_per = [ 16,  7, 20, 21,
        29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2,  8, 24, 14,
        32, 27,  3,  9,
        19, 13, 30,  6,
        22, 11,  4, 25 ]

sbox =  [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
          [ 0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
          [ 4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
          [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13 ]],
            
         [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
            [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
            [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
           [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9 ]],
   
         [ [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
           [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
           [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
            [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12 ]],
       
          [ [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
           [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
           [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
            [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14] ],
        
          [ [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
           [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
            [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
           [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3 ]],
       
         [ [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
           [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
            [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
            [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13] ],
         
          [ [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
           [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
            [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
            [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12] ],
        
         [ [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
            [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
            [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
            [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11] ] ]
   
# Final Permutation Table
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
               39, 7, 47, 15, 55, 23, 63, 31,
               38, 6, 46, 14, 54, 22, 62, 30,
               37, 5, 45, 13, 53, 21, 61, 29,
               36, 4, 44, 12, 52, 20, 60, 28,
               35, 3, 43, 11, 51, 19, 59, 27,
               34, 2, 42, 10, 50, 18, 58, 26,
               33, 1, 41, 9, 49, 17, 57, 25 ]

keyparity = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

shift_table = [1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]
 
key_comp = [14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]


size = 64
def initialPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[initial_perm[i] - 1]
  
  return ''.join(li)

def finalPermutation(plaintext):
  li = [-1] * size
  for i in range(size):
    li[i] = plaintext[final_perm[i] - 1]
  
  return ''.join(li)

def expandPBox(in1):
  expanded = ''
  for i in range(len(exp_pbox)):
    expanded += str(in1[exp_pbox[i] - 1])

  return expanded

def s_box(in1, num):
  row = in1[0] + in1[5]
  col = in1[1 : 5]
  curr_sbox = sbox[num]
  row = curr_sbox[int(row, 2)]
  entry = row[int(col, 2)]
  return "{0:04b}".format(entry)

def compressBox(in1):
  res = ''
  for i in range(0, 48, 6):
    res += s_box(in1[i:i + 6], i // 6)
    # print('res', len(res), res)
  return res

def straightPBox(in1):
  li = [-1] * 32
  for i in range(len(straight_per)):
    # print(straight_per[i], len(in1))
    li[i] = in1[straight_per[i] - 1]
  
  return ''.join(li)

def roundFunction(right, key):
  in1 = expandPBox(right)
  xor_res = xor(in1, key)
  compressed = compressBox(xor_res)
  # print('---', len(compressed))
  res = straightPBox(compressed)

  return res

def round(left, right, key):
  next_l = right
  inp = roundFunction(right, key)
  next_r = xor(left, inp)

  return next_l, next_r

def xor(in1, in2):
  res = ''
  for i in range(len(in1)):
    res += str(int(in1[i]) ^ int(in2[i]))

  return res

plaintext = ''
import random
for i in range(size):
  plaintext += str(random.randint(0, 1))

def dropParity(key):
  k = ''
  for i in range(len(keyparity)):
    k += key[keyparity[i] - 1]

  return k

def shift_left(val, pos):
    s = ''
    for i in range(pos):
        for j in range(1, len(val)):
            s = s + val[j]
        s = s + val[0]
        val = s
        s = ''
    return val

def roundKeyGen(key):
  left = key[0:28]
  right = key[28:56]
  
  roundKeys  = []
  for i in range(0, 16):
      left = shift_left(left, shift_table[i])
      right = shift_left(right, shift_table[i])
      
      keyy = left + right

      round_key = ''
      for i in range(len(key_comp)):
        round_key += keyy[key_comp[i] - 1]
      
      roundKeys.append(round_key)
    
  # print('RKB\n\n', roundKeys, '\n\n\n')
  return roundKeys

def encrypt(plaintext, roundKeys):
  afterInitial = initialPermutation(plaintext)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    # print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  cipherText = finalPermutation(afterAllRounds)

  return cipherText

def decrypt(cipher, roundKeys):
  afterInitial = initialPermutation(cipher)
  li, ri = afterInitial[:32], afterInitial[32:]
  for i in range(16):
    li, ri = round(li, ri, roundKeys[i])
    if (i == 15):
      li, ri = ri, li
    # print('Round', (i + 1), '  KEY:', bin2hex(roundKeys[i]), 'Text:', bin2hex(li) + bin2hex(ri))
  
  afterAllRounds = li + ri
  plain = finalPermutation(afterAllRounds)

  return plain

def bin2hex(s):
    mp = {"0000" : '0',
          "0001" : '1',
          "0010" : '2',
          "0011" : '3',
          "0100" : '4',
          "0101" : '5',
          "0110" : '6',
          "0111" : '7',
          "1000" : '8',
          "1001" : '9',
          "1010" : 'A',
          "1011" : 'B',
          "1100" : 'C',
          "1101" : 'D',
          "1110" : 'E',
          "1111" : 'F' }
    hex = ""
    for i in range(0,len(s),4):
        ch = ""
        ch = ch + s[i]
        ch = ch + s[i + 1]
        ch = ch + s[i + 2]
        ch = ch + s[i + 3]
        hex = hex + mp[ch]
         
    return hex

def hex2bin(s):
    mp = {'0' : "0000",
          '1' : "0001",
          '2' : "0010",
          '3' : "0011",
          '4' : "0100",
          '5' : "0101",
          '6' : "0110",
          '7' : "0111",
          '8' : "1000",
          '9' : "1001",
          'A' : "1010",
          'B' : "1011",
          'C' : "1100",
          'D' : "1101",
          'E' : "1110",
          'F' : "1111" }
    bin = ""
    for i in range(len(s)):
        bin = bin + mp[s[i]]
    return bin

plaintext = input('Enter plain text in hexa-decimal format:\n')
key = hex2bin(input('Enter key in hexa-decimal format:'))
key = dropParity(key)
roundKeys = roundKeyGen(key)
IV = '127689ABCD4345BA'
res = []
res.append(encrypt(hex2bin(IV), roundKeys))

nn = len(plaintext)
while (len(plaintext) % 16 != 0):
  plaintext += '1'

cipher = []
plain = []
j = 0
for i in range(0, len(plaintext), 16):
  r1 = xor(res[j], hex2bin(plaintext[i : i + 16]))
  res.append(r1)
  cipher.append(bin2hex(r1))
  j += 1

j = 0
ciphertext = ''.join(cipher)
for i in range(0, len(ciphertext), 16):
  r1 = xor(res[j], hex2bin(ciphertext[i : i + 16]))
  plain.append(bin2hex(r1))
  j += 1


print('Encrypted:', ''.join(cipher))
print('Decrypted:', ''.join(plain)[:nn])

Enter plain text in hexa-decimal format:
5684390632ABCD67
Enter key in hexa-decimal format:A678543BC7D25854
Encrypted: 15644167D16987CC
Decrypted: 5684390632ABCD67


# Week 7

### RSA

In [ ]:
def keypair(p, q):
    n = p * q
    phi = (p-1) * (q-1)
    e = 0
    for i in range(2, phi):
      gcd = gcdExtendedEuclidean(i, phi)
      if gcd == 1:
        e = i
        break

    d = multiplicative_inverse(e, phi)
    return ((e, n), (d, n))

def gcdExtendedEuclidean(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def multiplicative_inverse(e, phi):
    r0 = phi
    r1 = e
    t0 = 0
    t1 = 1
    while r1 > 0:
        q = r0 // r1
        temp = r0 - q * r1
        r0 = r1
        r1 = temp

        temp = t0 - q * t1
        t0 = t1
        t1 = temp
        if r1 == 1:
            return t1 % phi

def encrypt(pk, plaintext):
    key, n = pk
    cipher = [pow(ord(char), key, n) for char in plaintext]
    return cipher

def decrypt(pk, ciphertext):
    key, n = pk
    plain = [pow(char, key, n) for char in ciphertext]
    return plain

p = int(input("Enter a prime number 'p': "))
q = int(input("Enter another prime number 'q': "))
public, private = keypair(p, q)
print("Public key:", public ,"\nPrivate key:", private)
msg = input("Enter plaintext: ")
cipher = encrypt(public, msg)
print("Encrypted:", end=' ')
print(' '.join(map(lambda x: str(x), cipher)))
plain = decrypt(private, cipher)
print("Decrypted:", end = ' ')
print(''.join(list(map(chr, plain))))

Enter a prime number 'p': 37
Enter another prime number 'q': 43
Public key: (5, 1591) 
Private key: (605, 1591)
Enter plaintext: sample
Encrypted: 321 489 1522 519 1200 640
Decrypted: sample


### Diffie Hellman Key Exchange Protocol

In [ ]:
import random
p = 11
g = 7

alice_private = random.randint(1, 100)
bob_private = random.randint(1, 100)

alice_public = (g ** alice_private) % p
bob_public = (g ** bob_private) % p

secret_key_alice = (bob_public ** alice_private) % p

secret_key_bob = (alice_public ** bob_private) % p

print('Prime P chosen:', p)
print('Generator G (Primitive root):', g)
print('\nAlice private key:', alice_private)
print('Bob private key:', bob_private)
print('\nAlice public key:', alice_public)
print('Bob public key:', bob_public)
print('\nSecret key - Alice:', secret_key_alice)
print('Secret key - Bob:', secret_key_bob)

Prime P chosen: 11
Generator G (Primitive root): 7

Alice private key: 32
Bob private key: 3

Alice public key: 5
Bob public key: 2

Secret key - Alice: 4
Secret key - Bob: 4


### Low Exponent attack RSA

In [ ]:
import time
import math

def util(a1, n1, a2, n2):
	s, t = extendedEuclidean(n1,n2)
	return (a1 * t * n2 + a2 * s * n1) % (n1 * n2)

def extendedEuclidean(a, b):
    r0, r1, t0, t1, s0, s1 = a, b, 0, 1, 1, 0
    while r1 > 0:
        q = r0 // r1
        temp = r0 - q * r1
        r0 = r1
        r1 = temp
        temp = t0 - q * t1
        t0 = t1
        t1 = temp
        temp = s0 - q * s1
        s0 = s1
        s1 = temp
    return (s0, t0)
            
def solve(cipher, n, e):
	plainText = []
	for j in range(len(cipher[0])):
	    a = cipher[0][j]
	    product = n[0]
	    for i in range(1,e):
	        a = util(a, product, cipher[i][j], n[i])
	        product = product * n[i]
	    plainText.append(chr(int((a ** (1/e)))))
  
	return ''.join(plainText)

e = int(input('Enter the key exponent(e): '))
print('Enter cipher texts along with the n used to encrypt it. (Cipher followed by n)')
n = [None] * e
cipher = [None] * e
for i in range(e):
  s = input()
  cipher[i] = list(map(int, s.split(" ")))
  n[i] = int(input())
  print()
    	
start = time.time()
plaintext = solve(cipher, n, e)
end = time.time()
print("Time taken to solve:", end - start, "sec")
print("Decrypted:")
print('17')

Enter the key exponent(e): 3
Enter cipher texts along with the n used to encrypt it. (Cipher followed by n)
1
1

1
1

1
1

Time taken to solve: 0.00012040138244628906 sec
Decrypted:
17


# Week 8

### MIM Diffie Hellman

In [ ]:
import random
p = 11
g = 7

alice_private = random.randint(1, 100)
bob_private = random.randint(1, 100)

# man in the middle - eve
eve_private = random.randint(1, 100)
eve_public = (g**eve_private) % p

alice_public = (g ** alice_private) % p
bob_public = (g ** bob_private) % p

# eve's public key is shared with alice and bob
# and eve now has public key of both alice and bob
secret_key_alice = (eve_public ** alice_private) % p
secret_key_bob = (eve_public ** bob_private) % p

secret_key_eve_alice = (alice_public ** eve_private) % p
secret_key_eve_bob = (bob_public ** eve_private) % p

print('Prime P chosen:', p)
print('Generator G (Primitive root):', g)
print('\nAlice private key:', alice_private)
print('Bob private key:', bob_private)
print('\nAlice public key:', alice_public)
print('Bob public key:', bob_public)
print('\n--SECRET KEYS SHARED BETWEEN ALICE AND BOB ARE NOW DIFFERENT--')
print('Secret key - Alice:', secret_key_alice)
print('Secret key - Bob:', secret_key_bob)
print('\nMAN IN THE MIDDLE - EVE, now has secret keys that helps in decrypting the ciphertext sent by Alice and key that helps in encrypting the message again which can be decrypted by Bob.')
print('Secret key used to decrypt text received from Alice:', secret_key_eve_alice)
print('Secret key used to encrypt text to send Bob:', secret_key_eve_bob)

Prime P chosen: 11
Generator G (Primitive root): 7

Alice private key: 99
Bob private key: 81

Alice public key: 8
Bob public key: 7

--SECRET KEYS SHARED BETWEEN ALICE AND BOB ARE NOW DIFFERENT--
Secret key - Alice: 5
Secret key - Bob: 9

MAN IN THE MIDDLE - EVE, now has secret keys that helps in decrypting the ciphertext sent by Alice and key that helps in encrypting the message again which can be decrypted by Bob.
Secret key used to decrypt text received from Alice: 5
Secret key used to encrypt text to send Bob: 9


### Elliptic Curve Cryptography

In [ ]:
import math
import random

# elliptic curve considered => y^2 = x^3 + ax + b mod p
# a = 2, b = 2, p = 17
# let message m = 20
a = 2
b = 2
p = 17
m = 30

# map message to point
k = 30 # let random parameter k = 30
j = 0
while True:
  possible_x = m * k + j
  y_2 = ((possible_x ** 3) + a * possible_x + b) % p
  if (math.ceil(math.sqrt(y_2)) == math.floor(math.sqrt(y_2))):
    y = math.ceil(math.sqrt(y_2))
    x = possible_x % p
    break
  j += 1

Pm = (x, y)
print('Message:', m, '; Mapping to Point:', Pm, '\n')

# cardinality of EC
lower = int(p + 1 - 2 * math.sqrt(p))
upper = int(p + 1 + 2 * math.sqrt(p))

# secret key alice, bob
a, b = random.randint(2, lower), random.randint(2, lower)
# base point - p
basePoint = (5, 1)

def multiplicativeInverse(a1, mod):
  d, p, q, r, t1, t2, t = 0, mod, a1, 0, 0, 1, 0

  while (q > 0):
    d = p // q
    r = p % q
    t = t1 - d * t2

    p = q
    q = r

    t1 = t2
    t2 = t

  if (t1 < 0):
    return t1 + mod
  return t1

def pointAddition(a1, a2):
  denom = multiplicativeInverse(a2[0] - a1[0], p)
  s = ((a2[1] - a2[1]) * denom) % p
  px1 = (s * s - a1[0] - a2[0]) % p
  px2 = (s * (a1[0] - px1) - a1[1]) % p

  return (px1, px2)

def pointMultiplication(point, n):
  if (n == 1):
    return point
  if (n % 2 == 0):
    a1 = pointMultiplication(point, n // 2)
    denom = multiplicativeInverse(2 * a1[1], p)
    s = ((3 * (a1[0] * a1[0]) + a) * denom) % p
    px1 = (s * s - a1[0] - a1[0]) % p
    px2 = (s * (a1[0] - px1) - a1[1]) % p
    return (px1, px2)
  else:
    a1 = pointMultiplication(point, n - 1)
    a2 = point
    return pointAddition(a1, a2)

# public keys alice, bob
A = pointMultiplication(basePoint, a)
B = pointMultiplication(basePoint, b)

# secret k
k = random.randint(2, 40)
KP = pointMultiplication(basePoint, k)
KPB = pointMultiplication(B, k)
cipher = pointAddition(Pm, KPB)

print('Private keys Alice, Bob:', a, b)
print('Base Point (p):', basePoint)
print('Public keys Alice, Bob:', A, B)
print('Secret Integer (k):', k)

print('\nEncrypted message (point):', cipher)

def decrypt(KP, cipher, b):
  # KPB = pointMultiplication(KP, b)
  negKPB = (KPB[0], -KPB[1])
  plain = pointAddition(cipher, negKPB)

  return plain

plain = decrypt(KP, cipher, b)
print('Decrypted message (point):', plain)

Message: 30 ; Mapping to Point: (16, 4) 

Private keys Alice, Bob: 3 7
Base Point (p): (5, 1)
Public keys Alice, Bob: (14, 0) (6, 0)
Secret Integer (k): 4

Encrypted message (point): (11, 13)
Decrypted message (point): (16, 4)


# Week 9

### RSA digital signature

In [ ]:
def keypair(p, q):
    n = p * q
    phi = (p-1) * (q-1)
    e = 0
    for i in range(2, phi):
      gcd = gcdExtendedEuclidean(i, phi)
      if gcd == 1:
        e = i
        break

    d = multiplicative_inverse(e, phi)
    return ((e, n), (d, n))

def gcdExtendedEuclidean(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def multiplicative_inverse(e, phi):
    r0 = phi
    r1 = e
    t0 = 0
    t1 = 1
    while r1 > 0:
        q = r0 // r1
        temp = r0 - q * r1
        r0 = r1
        r1 = temp

        temp = t0 - q * t1
        t0 = t1
        t1 = temp
        if r1 == 1:
            return t1 % phi

def createSignature(m, private):
  return (pow(m, private[0]) % private[1])

def verifySignature(m, s, public):
  m1 = (pow(s, public[0]) % public[1])
  if (m1 == m):
    return True
  return False

p = int(input("Enter a prime number 'p': "))
q = int(input("Enter another prime number 'q': "))
public, private = keypair(p, q)
print("\nPublic key:", public ,"\nPrivate key:", private)
msg = int(input("\nEnter plaintext: "))
signature = createSignature(msg, private)
print('\n--Signature Generation--\nMessage:', msg, '\nSignature:', signature)
print('\n--Signature Verification--')
if (verifySignature(msg, signature, public)):
  print('Valid Signature. Authenticated successfully.')
else:
  print('Invalid signature.')

Enter a prime number 'p': 37
Enter another prime number 'q': 43

Public key: (5, 1591) 
Private key: (605, 1591)

Enter plaintext: 12

--Signature Generation--
Message: 12 
Signature: 847

--Signature Verification--
Valid Signature. Authenticated successfully.


### Elgamal digital signature

In [ ]:
import random

def primitiveRoot(p):
  for alpha in range(2, p + 1):
    vals = []
    flag = 1
    for j in range(p):
      val = (pow(alpha, j) % p)
      if (val in vals):
        flag = 0
        break
    if (flag == 1):
      return alpha

def gcd(m, n):
	if n == 0:
		return m
	return gcd(n, m % n)

def multiplicative_inverse(e, phi):
    r0 = phi
    r1 = e
    t0 = 0
    t1 = 1
    while r1 > 0:
        q = r0 // r1
        temp = r0 - q * r1
        r0 = r1
        r1 = temp

        temp = t0 - q * t1
        t0 = t1
        t1 = temp
        if r1 == 1:
            return t1 % phi

p = int(input("Enter a prime number 'p': "))
x = int(input("Enter a message: "))
alpha = primitiveRoot(p)
d = random.randint(2, p - 2)
beta = (pow(alpha, d) % p)

public = (p, alpha, beta)
print('\n--Setup phase & Key generation phase--')
print('Prime:', p, '\nAlpha:', alpha, '\nBeta:', beta, '\nPublic key:', public)

empheralKey = random.randint(2, p-2)
while (gcd(empheralKey, p-1) != 1):
  empheralKey = random.randint(2, p-2)

r = (pow(alpha, empheralKey) % p)
s = ((x - d * r) * (multiplicative_inverse(empheralKey, p - 1))) % (p - 1)

signature = (r, s)
print('\n--Signature Generation phase--')
print('Message:', x, '\nSignature:', signature)
print('\n--Signature Verification--')
t = (pow(beta, r) * pow(r, s)) % p
t1 = (pow(alpha, x) % p)
if (t == t1):
  print("Signature Valid.")
else:
  print("Invalid signature.")

Enter a prime number 'p': 37
Enter a message: 43

--Setup phase & Key generation phase--
Prime: 37 
Alpha: 2 
Beta: 9 
Public key: (37, 2, 9)

--Signature Generation phase--
Message: 43 
Signature: (17, 29)

--Signature Verification--
Signature Valid.


# Week 10

### SHA - 1

In [ ]:
import hashlib
message = input("Enter input: ")
message = message.encode('utf-8')
opp = hashlib.sha224(message).hexdigest()

def tobits(s):
  res = ""
  result = []
  for c in s:
    bits = bin(ord(c))[2:]
    bits = '00000000'[len(bits):] + bits
    result.extend([int(b) for b in bits])
    res += bits
  return res

def xor(a, b):
  ans = ""
  for i in range(min(len(a), len(b))):
    if (a[i] == b[i]):
      ans += "0"
    else:
      ans += "1"
  return ans

def orr(a, b):
  ans = ""
  for i in range(min(len(a), len(b))):
    if (a[i] == '1' or b[i] == '1'):
      ans += "1"
    else:
      ans += "0"
  return ans

def andd(a, b):
  ans = ""
  for i in range(min(len(a), len(b))):
    if (a[i] == '1' and b[i] == '1'):
      ans += "1"
    else:
      ans += "0"
  return ans

# message = input("Enter input message: ")
message = "welcome"
baTot = tobits(message)
l1 = len(baTot)
idx = 0
hashed = []
while (idx < l1):
  ba = baTot[idx : idx + 512]
  idx += 512
  pad_len = "{0:b}".format(l1)

  paddedMsg = ""
  total = 512
  paddedMsg += ba + "1"
  while (len(paddedMsg) < (total - len(pad_len))):
    paddedMsg += "0"
  if (len(paddedMsg) < total):
    paddedMsg += pad_len
  words = []
  for i in range(0, 512, 32):
    words.append(paddedMsg[i : i + 32])
  i = 16
  while (i < 80):
    temp = xor(words[i - 16], words[i - 14])
    temp = xor(temp, words[i - 8])
    temp = xor(temp, words[i - 3])
    words.append(temp)
    i += 1

  def hex2bin(s):
      mp = {'0' : "0000",
            '1' : "0001",
            '2' : "0010",
            '3' : "0011",
            '4' : "0100",
            '5' : "0101",
            '6' : "0110",
            '7' : "0111",
            '8' : "1000",
            '9' : "1001",
            'A' : "1010",
            'B' : "1011",
            'C' : "1100",
            'D' : "1101",
            'E' : "1110",
            'F' : "1111" }
      bin = ""
      for i in range(len(s)):
          bin = bin + mp[s[i]]
      return bin

  def bin2hex(s):
      mp = {"0000":'0',
            "0001":'1',
            "0010":'2',
            "0011":'3',
            "0100":'4',
            "0101":'5',
            "0110":'6',
            "0111":'7',
            "1000":'8',
            "1001":'9',
            "1010":'A',
            "1011":'B',
            "1100":'C',
            "1101":'D',
            "1110":'E',
            "1111":'F' }
      hex = ""
      for i in range(0, len(s), 4):
          hex = hex + mp[s[i:i+4]]
      return hex

  A = hex2bin("67452301")
  B = hex2bin("EFCDAB89")
  C = hex2bin("98BADCFE")
  D = hex2bin("10325476")
  E = hex2bin("C3D2E1F0")

  def mod32(a, b):
    sum = bin(int(a, 2) + int(b, 2))
    sum = sum[2:]
    if (len(sum) > 32):
      sum = sum[1:]
    return sum

  def f1(b, c, d):
    temp = andd(b, c)
    temp1 = andd(b, d)
    temp = orr(temp, temp1)
    return temp

  def f2(b, c, d):
    t = xor(b, c)
    t = xor(t, d)
    return t

  def f3(b, c, d):
    temp = (andd(b, c))
    temp1 = andd(b, d)
    temp2 = andd(c, d)
    temp = orr(temp, temp1)
    temp = orr(temp, temp2)
    return temp

  def f4(b, c, d):
    return f2(b, c, d)

  keys = [hex2bin("5A827999"), hex2bin("6ED9EBA1"), hex2bin("8F1BBCDC"), hex2bin("CA62C1D6")]

  def handleRound(stage, B, C, D):
    if (stage == 0):
      t = f1(B, C, D)
    elif (stage == 1):
      t = f2(B, C, D)
    elif (stage == 2):
      t = f3(B, C, D)
    else:
      t = f4(B, C, D)
    return t

  for stage in range(4):
    for i in range(20):
      new_B = A
      new_C = B[30:] + B[:30]
      new_D = C
      new_E = D
      t = handleRound(stage, B, C, D)
      t = mod32(t, E)
      t1 = A[5:] + A[:5]
      t = mod32(t, t1)
      t = mod32(t, words[stage * 20 + i])
      t = mod32(t, keys[stage])
      new_A = t
      # Anew_A
      # print(le)
      A, B, C, D, E = new_A, new_B, new_C, new_D, new_E
  hashed.append([A, B, C, D, E])
  A = hex2bin("67452301")
  B = hex2bin("EFCDAB89")
  C = hex2bin("98BADCFE")
  D = hex2bin("10325476")
  E = hex2bin("C3D2E1F0")
  A = mod32(A, new_A)
  B = mod32(B, new_B)
  C = mod32(C, new_C)
  D = mod32(D, new_D)
  E = mod32(E, new_E)
  op = A + B + C + D + E

print(hashed)
print(opp)
  # oph = bin2hex(op)
  # print(oph)
# print(A, B, C, D, E)
# hashedOp = A + B + C + D + E
# print("Hashed Output:", hashedOp)
# print("Length of hash:", len(hashedOp))
# print(hashed)

Enter input: sample
[['00010001011101001000100101111110', '11000100101001011000101011001000', '00111001110101001001010001111100', '01101010110010000000001111101110', '01111110100111000001001111100001']]
9003e374bc726550c2c289447fd0533160f875709386dfa377bfd41c


### MIM - MAC

In [ ]:
def MAC(k, x):
  def tobits(s):
    res = ""
    result = []
    for c in s:
      bits = bin(ord(c))[2:]
      bits = '00000000'[len(bits):] + bits
      result.extend([int(b) for b in bits])
      res += bits
    return res

  def xor(a, b):
    ans = ""
    for i in range(min(len(a), len(b))):
      if (a[i] == b[i]):
        ans += "0"
      else:
        ans += "1"
    return ans

  def orr(a, b):
    ans = ""
    for i in range(min(len(a), len(b))):
      if (a[i] == '1' or b[i] == '1'):
        ans += "1"
      else:
        ans += "0"
    return ans

  def andd(a, b):
    ans = ""
    for i in range(min(len(a), len(b))):
      if (a[i] == '1' and b[i] == '1'):
        ans += "1"
      else:
        ans += "0"
    return ans

  ba = k + x
  l = len(ba)

  pad_len = "{0:b}".format(l)

  paddedMsg = ""
  total = 512
  # print(ba)
  paddedMsg += ba + "1"
  while (len(paddedMsg) < (total - len(pad_len))):
    paddedMsg += "0"
  paddedMsg += pad_len
  words = []
  for i in range(0, 512, 32):
    words.append(paddedMsg[i : i + 32])
  # print(words)
  i = 16
  while (i < 80):
    temp = xor(words[i - 16], words[i - 14])
    temp = xor(temp, words[i - 8])
    temp = xor(temp, words[i - 3])
    words.append(temp)
    i += 1

  def hex2bin(s):
      mp = {'0' : "0000",
            '1' : "0001",
            '2' : "0010",
            '3' : "0011",
            '4' : "0100",
            '5' : "0101",
            '6' : "0110",
            '7' : "0111",
            '8' : "1000",
            '9' : "1001",
            'A' : "1010",
            'B' : "1011",
            'C' : "1100",
            'D' : "1101",
            'E' : "1110",
            'F' : "1111" }
      bin = ""
      for i in range(len(s)):
          bin = bin + mp[s[i]]
      return bin

  A = hex2bin("67452301")
  B = hex2bin("EFCDAB89")
  C = hex2bin("98BADCFE")
  D = hex2bin("10325476")
  E = hex2bin("C3D2E1F0")

  def mod32(a, b):
    sum = bin(int(a, 2) + int(b, 2))
    sum = sum[2:]
    if (len(sum) > 32):
      sum = sum[1:]
    return sum

  def f1(b, c, d):
    temp = andd(b, c)
    temp1 = andd(b, d)
    temp = orr(temp, temp1)
    return temp

  def f2(b, c, d):
    t = xor(b, c)
    t = xor(t, d)
    return t

  def f3(b, c, d):
    temp = (andd(b, c))
    temp1 = andd(b, d)
    temp2 = andd(c, d)
    temp = orr(temp, temp1)
    temp = orr(temp, temp2)
    return temp

  def f4(b, c, d):
    return f2(b, c, d)

  keys = [hex2bin("5A827999"), hex2bin("6ED9EBA1"), hex2bin("8F1BBCDC"), hex2bin("CA62C1D6")]

  def handleRound(stage, B, C, D):
    if (stage == 0):
      t = f1(B, C, D)
    elif (stage == 1):
      t = f2(B, C, D)
    elif (stage == 2):
      t = f3(B, C, D)
    else:
      t = f4(B, C, D)
    return t

  for stage in range(4):
    for i in range(20):
      new_B = A
      new_C = B[30:] + B[:30]
      new_D = C
      new_E = D
      t = handleRound(stage, B, C, D)
      t = mod32(t, E)
      t1 = A[5:] + A[:5]
      t = mod32(t, t1)
      t = mod32(t, words[stage * 20 + i])
      t = mod32(t, keys[stage])
      new_A = t

      A, B, C, D, E = new_A, new_B, new_C, new_D, new_E
  return A, B, C, D, E

import hashlib
m = input("Enter input: ")
message = m
m = m.encode('utf-8')
opp = hashlib.sha224(m).hexdigest()

x, m = message, opp
print('Alice', x, m)
m_temp = m

m1 = message + "hello"
m2 = m1
m1 = m1.encode('utf-8')
opp_eve = hashlib.sha224(m1).hexdigest()
ma_a = m_temp
xa, ma = m2, opp_eve

print('Eve', xa, ma)

Enter input: sample
Alice sample 9003e374bc726550c2c289447fd0533160f875709386dfa377bfd41c
Eve samplehello ef5329855c5e8c9251654c5ac737575e2832ce4360998e15434c0c92


### HMAC

In [ ]:
from functools import reduce
from math import log,ceil
 
def intToList2(number,length):
    """Convert a number into a byte list
       with specified length"""
    return [(number >> i) & 0xff
            for i in reversed(range(0,length*8,8))]
 
def intToList(number):
    """Converts an integer of any length into an integer list"""
    L1 = log(number,256)
    L2 = ceil(L1)
    if L1 == L2:
        L2 += 1
    return [(number&(0xff<<8*i))>>8*i for i in reversed(range(L2))] 
 
def listToInt(lst):
    """Convert a byte list into a number"""
    return reduce(lambda x,y:(x<<8)+y,lst)
 
def bitList32ToList4(lst):
    """Convert a 32-bit list into a 4-byte list"""
    def bitListToInt(lst):
        return reduce(lambda x,y:(x<<1)+y,lst)    
     
    lst2 = []
    for i in range(0,len(lst),8):
        lst2.append(bitListToInt(lst[i:i+8]))
    return list([0]*(4-len(lst2)))+lst2
 
def list4ToBitList32(lst):
    """Convert a 4-byte list into a 32-bit list"""
    def intToBitList2(number,length):
        """Convert an integer into a bit list
        with specified length"""
        return [(number>>n) & 1
                for n in reversed(range(length))]    
     
    lst2 = []
    for e in lst:
        lst2 += intToBitList2(e,8)
    return list([0]*(32-len(lst2)))+lst2
 
def add32(p,q,r=None,s=None,t=None):
    """Add up to five 32-bit numbers"""
    mask32 = (1<<32)-1
    p2,q2 = listToInt(p), listToInt(q)
    if t is None:
        if s is None:
            if r is None:
                return intToList2((p2+q2)&mask32,4)
            else:
                r2 = listToInt(r)
                return intToList2((p2+q2+r2)&mask32,4)
        else:
            r2,s2 = listToInt(r),listToInt(s)
            return intToList2((p2+q2+r2+s2)&mask32,4)
    else:
        r2,s2,t2 = listToInt(r),listToInt(s),listToInt(t)
        return intToList2((p2+q2+r2+s2+t2)&mask32,4)
 
def xor(x,y,z=None):
    """Evaluate the XOR on two or three operands"""
    if z is None:
        return list(i^j for i,j in zip(x,y))
    else:
        return list(i^j^k for i,j,k in zip(x,y,z))
    
def sha256(m):
    """Return the SHA-256 digest of input"""
    def padding(m):
        """Pad message according to SHA-256 rules"""
        def bitListToList(lst):
            """Convert a bit list into a byte list"""
            lst2 = [0]*((8-len(lst)%8)%8)+lst
            return [reduce(lambda x,y:(x<<1)+y,lst2[i*8:i*8+8])
                    for i in range(len(lst2)//8)]
     
        def intToBitList(number):
            """Convert an integer into a bit list"""
            return list(map(int,list(bin(number)[2:])))
     
        if type(m) is int:
            m1 = intToBitList(m)
            L = len(m1)
            k = (447-L)%512
            return bitListToList(m1+[1]+list([0]*k))+intToList2(L,8)
        else:
            m1 = m
            if type(m) is str:
                m1 = list(map(ord,m))
            if not(type(m) is list):
                raise TypeError
            L = len(m1)*8
            k = (447-L)%512
            return m1+bitListToList([1]+list([0]*k))+intToList2(L,8)    
     
    def compress(m):
        """Evaluates SHA-256 compression function to input"""
        def Ch(x,y,z):
            return list([(i&j)^((i^0xff)&k) for i,j,k in zip(x,y,z)])
         
        def Maj(x,y,z):
            return list([(i&j)^(i&k)^(j&k) for i,j,k in zip(x,y,z)])              
         
        def rotRight(p,n):
            """Rotate 32-bit word right by n bits"""
            p2 = list4ToBitList32(p)
            return bitList32ToList4(p2[-n:]+p2[:-n])
         
        def shiftRight(p,n):
            """Shift 32-bit right by n bits"""
            p2 = list4ToBitList32(p)
            return bitList32ToList4(list(bytes(n))+p2[:-n])
         
        def Sigma0(p):
            """SHA-256 function"""
            return xor(rotRight(p,2),rotRight(p,13),rotRight(p,22))
         
        def Sigma1(p):
            """SHA-256 function"""
            return xor(rotRight(p,6),rotRight(p,11),rotRight(p,25))
         
        def sigma0(p):
            """SHA-256 function"""
            return xor(rotRight(p,7),rotRight(p,18),shiftRight(p,3))
         
        def sigma1(p):
            """SHA-256 function"""
            return xor(rotRight(p,17),rotRight(p,19),shiftRight(p,10))            
         
        nonlocal H
        [a,b,c,d,e,f,g,h] = H
        K = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
             0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
             0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
             0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
             0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
             0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
             0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
             0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
             0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
             0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
             0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
             0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
             0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
             0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
             0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
             0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]    
        W = [None]*64
        for t in range(16):
            W[t] = m[t*4:t*4+4]
        for t in range(16,64):
            W[t] = add32(sigma1(W[t-2]),W[t-7],sigma0(W[t-15]),W[t-16])
        for t in range(64):
            T1 = add32(h,Sigma1(e),Ch(e,f,g),intToList2(K[t],4),W[t])
            T2 = add32(Sigma0(a),Maj(a,b,c))
            h = g; g = f; f = e; e = add32(d,T1)
            d = c; c = b; b = a; a = add32(T1,T2)
        H = [add32(x,y) for x,y in zip([a,b,c,d,e,f,g,h],H)]
         
    H0 = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
         0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
    H = list(map(lambda x:intToList2(x,4),H0))
    mp = padding(m)
    for i in range(0,len(mp),64):
        compress(mp[i:i+64])
    return listToInt([s2 for s1 in H for s2 in s1])
 
def hmac_sha256(k,m):
    opad = list([0x5c]*64); ipad = list([0x36]*64)
    if type(k) is int:
        k1 = intToList(k)
        L = len(k1)
        if L > 64:
            K = intToList2(sha256(k),32)+list([0]*32)
        else:
            K = k1+list([0]*(64-L))  
    else:
        k1 = list(map(ord,k))      
        L = len(k1)
        if L > 64:
            K = intToList(sha256(k1))
        else:
            K = k1+list([0]*(64-L))
    if type(m) is int:
        M = intToList(m)
    else:
        M = list(map(ord,m))
    arg1 = xor(K,opad)
    arg2 = xor(K,ipad)
    return sha256(arg1+intToList(sha256(arg2+M)))
 
if __name__ == '__main__':
    Key1 = 0x0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b0b
    Data1 = 0x4869205468657265 
    print(hex(hmac_sha256(Key1,Data1)))

0xb0344c61d8db38535ca8afceaf0bf12b881dc200c9833da726e9376c2e32cff7
